# 1. DBBE

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics import adjusted_rand_score, v_measure_score
from tqdm import tqdm
from collections import defaultdict
import igraph as ig
import leidenalg as la
import faiss
import os
from typing import Dict
import re
import unicodedata
import pandas as pd
from collections import defaultdict
from itertools import combinations

os.makedirs('dbbe_semantic_results', exist_ok=True)

csv_path = 'dbbe_full.csv'
df = pd.read_csv(csv_path)
df = df.dropna()

model_name = 'kevinkrahn/shlm-grc-en'
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

def cls_pooling(model_output):
    return model_output[0][:, 0]

verses = df['verse'].tolist()
embeddings = []
batch_size = 32

for i in tqdm(range(0, len(verses), batch_size), desc="Embeddings"):
    batch = verses[i:i+batch_size]
    encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    
    with torch.no_grad():
        model_output = model(**encoded_input)
    
    batch_embeddings = cls_pooling(model_output).cpu().numpy()
    embeddings.append(batch_embeddings)

embeddings = np.vstack(embeddings)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

dimension = embeddings.shape[1]
nlist = 500
k = 50
nprobe = 10

quantizer = faiss.IndexFlatIP(dimension)
index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_INNER_PRODUCT)

index.train(embeddings.astype('float32'))
index.add(embeddings.astype('float32'))
index.nprobe = nprobe
distances, indices = index.search(embeddings.astype('float32'), k)

similarity_thresholds = [0.70, 0.75, 0.80, 0.85, 0.90]
threshold_results = []
all_resolution_results = []

for sim_threshold in similarity_thresholds:
    print(f"\n{'='*80}")
    print(f"Threshold: {sim_threshold:.2f}")
    print(f"{'='*80}")
    
    candidate_pairs = set()
    
    for i in tqdm(range(len(embeddings)), desc=f"Building graph (t={sim_threshold:.2f})"):
        for j_idx, distance in zip(indices[i], distances[i]):
            if j_idx != i and j_idx != -1:
                similarity = distance
                if similarity >= sim_threshold:
                    idx1, idx2 = min(i, j_idx), max(i, j_idx)
                    candidate_pairs.add((idx1, idx2))
    
    n_pairs = len(candidate_pairs)
    avg_degree = n_pairs * 2 / len(embeddings)
    
    print(f"Candidate pairs: {n_pairs:,} (avg degree: {avg_degree:.1f})")
    
    if n_pairs == 0:
        print("No pairs found - skipping")
        threshold_results.append({
            'threshold': sim_threshold,
            'n_pairs': 0,
            'best_resolution': None,
            'best_ari': 0,
            'best_v_measure': 0,
            'n_clusters': 0
        })
        continue
    
    edges = []
    weights = []
    
    for i, j in tqdm(candidate_pairs, desc="Edge weights"):
        sim = float(np.dot(embeddings[i], embeddings[j]))
        edges.append((i, j))
        weights.append(sim)
    
    g = ig.Graph(n=len(embeddings), edges=edges, directed=False)
    g.es['weight'] = weights
    
    print(f"Graph: {g.vcount()} nodes, {g.ecount()} edges")
    
    w = np.array(weights)
    w_scaled = ((w - w.min()) / (w.max() - w.min())) ** 3
    g.es['weight'] = w_scaled.tolist()
    
    hub_thresh = 500
    for v in range(g.vcount()):
        if g.degree(v) > hub_thresh:
            for e in g.incident(v):
                g.es[e]['weight'] *= 0.5
    
    print("Leiden resolution sweep...")
    resolutions = np.logspace(-2, 1, 20)
    
    best_ari = -1
    best_labels = None
    best_res = None
    best_v = None
    
    for res in tqdm(resolutions, desc="Resolutions"):
        partition = la.find_partition(
            g,
            la.CPMVertexPartition,
            weights='weight',
            resolution_parameter=res,
            n_iterations=-1
        )
        labels = np.array(partition.membership)
        ari = adjusted_rand_score(df['idgroup'], labels)
        v_measure = v_measure_score(df['idgroup'], labels)
        n_clusters = len(set(labels))
        
        col_name = f'cluster_t{int(sim_threshold*100)}_r{res:.6f}'
        df[col_name] = labels
        
        all_resolution_results.append({
            'threshold': sim_threshold,
            'resolution': res,
            'ari': ari,
            'v_measure': v_measure,
            'n_clusters': n_clusters,
            'column_name': col_name
        })
        
        if ari > best_ari:
            best_ari = ari
            best_labels = labels
            best_res = res
            best_v = v_measure
    
    n_clusters = len(set(best_labels))
    
    print(f"Best resolution: {best_res:.4f}")
    print(f"Best ARI: {best_ari:.4f}")
    print(f"Best V-measure: {best_v:.4f}")
    print(f"Clusters: {n_clusters}")
    
    threshold_results.append({
        'threshold': sim_threshold,
        'n_pairs': n_pairs,
        'avg_degree': avg_degree,
        'best_resolution': best_res,
        'best_ari': best_ari,
        'best_v_measure': best_v,
        'n_clusters': n_clusters
    })
    
    df[f'cluster_t{int(sim_threshold*100)}_best'] = best_labels

print("\n" + "="*80)
print("GRID SEARCH RESULTS (BEST PER THRESHOLD)")
print("="*80)

results_df = pd.DataFrame(threshold_results)
print(results_df.to_string(index=False))

best_threshold_row = results_df.loc[results_df['best_ari'].idxmax()]
best_threshold = best_threshold_row['threshold']

print(f"\n{'='*80}")
print(f"OVERALL BEST THRESHOLD: {best_threshold:.2f}")
print(f"{'='*80}")
print(f"ARI: {best_threshold_row['best_ari']:.4f}")
print(f"V-measure: {best_threshold_row['best_v_measure']:.4f}")
print(f"Resolution: {best_threshold_row['best_resolution']:.4f}")
print(f"Clusters: {int(best_threshold_row['n_clusters'])}")

df['cluster_best'] = df[f'cluster_t{int(best_threshold*100)}_best']

df.to_csv("dbbe_semantic_results/faiss_leiden_gridsearch_results.csv", index=False)
results_df.to_csv("dbbe_semantic_results/threshold_gridsearch_summary.csv", index=False)

all_resolution_df = pd.DataFrame(all_resolution_results)
all_resolution_df.to_csv("dbbe_semantic_results/all_threshold_resolution_results.csv", index=False)

print("\nFiles saved:")
print("  - dbbe_semantic_results/faiss_leiden_gridsearch_results.csv")
print("  - dbbe_semantic_results/threshold_gridsearch_summary.csv")
print("  - dbbe_semantic_results/all_threshold_resolution_results.csv")

print(f"\nTotal clustering columns created: {len(all_resolution_results)}")
print(f"Thresholds tested: {len(similarity_thresholds)}")
print(f"Resolutions per threshold: {len(resolutions)}")

def calculate_jaccard_similarity(clusters_a, clusters_b):
    if not clusters_a or not clusters_b:
        return 0.0
    intersection = len(clusters_a & clusters_b)
    union = len(clusters_a | clusters_b)
    return intersection / union

def reconstruct_poems(df):
    poem_to_clusters = defaultdict(set)
    poem_verse_counts = defaultdict(int)

    for _, row in df.iterrows():
        poem_id = row['idoriginal_poem']
        cluster_id = row['cluster_leiden_fixed']
        poem_verse_counts[poem_id] += 1
        if cluster_id != -1:
            poem_to_clusters[poem_id].add(cluster_id)

    print(f"\nReconstructed {len(poem_to_clusters)} poems")
    return poem_to_clusters, poem_verse_counts

def evaluate_against_ground_truth(df, poem_clusters):
    poem_to_type = df.groupby('idoriginal_poem')['type_id'].first().to_dict()

    y_true = []
    y_pred = []
    for poem_id, predicted_cluster in poem_clusters.items():
        if poem_id in poem_to_type:
            y_true.append(poem_to_type[poem_id])
            y_pred.append(predicted_cluster)

    ari = adjusted_rand_score(y_true, y_pred)
    v_measure = v_measure_score(y_true, y_pred)

    return ari, v_measure, y_true, y_pred

def cluster_poems_jaccard(poem_to_clusters, similarity_threshold=0.66):
    poem_ids = list(poem_to_clusters.keys())
    n_poems = len(poem_ids)

    edges = []
    for i in range(n_poems):
        for j in range(i + 1, n_poems):
            pid_a, pid_b = poem_ids[i], poem_ids[j]
            sim = calculate_jaccard_similarity(poem_to_clusters[pid_a], poem_to_clusters[pid_b])
            if sim >= similarity_threshold:
                edges.append((pid_a, pid_b))

    class UF:
        def __init__(self, elements):
            self.parent = {e: e for e in elements}
            self.rank = {e: 0 for e in elements}

        def find(self, x):
            if self.parent[x] != x:
                self.parent[x] = self.find(self.parent[x])
            return self.parent[x]

        def union(self, x, y):
            px, py = self.find(x), self.find(y)
            if px == py: return
            if self.rank[px] < self.rank[py]: px, py = py, px
            self.parent[py] = px
            if self.rank[px] == self.rank[py]: self.rank[px] += 1

    uf = UF(poem_ids)
    for a, b in edges:
        uf.union(a, b)

    poem_clusters = {pid: uf.find(pid) for pid in poem_ids}
    return poem_clusters, edges

def calculate_perfect_reconstruction_rate(df, poem_clusters):
    poem_to_type = df.groupby('idoriginal_poem')['type_id'].first().to_dict()

    gt_to_poems = defaultdict(set)
    for poem_id, gt_type in poem_to_type.items():
        gt_to_poems[gt_type].add(poem_id)

    pred_to_poems = defaultdict(set)
    for poem_id, pred_cluster in poem_clusters.items():
        pred_to_poems[pred_cluster].add(poem_id)

    perfectly_reconstructed = 0
    total_gt_clusters = len(gt_to_poems)

    for gt_type, gt_poems in gt_to_poems.items():
        for pred_cluster, pred_poems in pred_to_poems.items():
            if gt_poems == pred_poems:
                perfectly_reconstructed += 1
                break

    reconstruction_rate = perfectly_reconstructed / total_gt_clusters if total_gt_clusters > 0 else 0
    return reconstruction_rate, perfectly_reconstructed, total_gt_clusters

df = pd.read_csv("dbbe_semantic_results/faiss_leiden_gridsearch_results.csv")

if 'cluster_best' in df.columns:
    df['cluster_leiden_fixed'] = df['cluster_best']
else:
    raise ValueError("Column 'cluster_best' not found in CSV")

poem_to_clusters, _ = reconstruct_poems(df)

thresholds = [0.50, 0.60, 0.70, 0.8]
results = []

for thresh in thresholds:
    print(f"\nThreshold {thresh:.0%}...")
    poem_clusters, edges = cluster_poems_jaccard(poem_to_clusters, thresh)
    df['poem_cluster_id'] = df['idoriginal_poem'].map(poem_clusters)

    ari, v_measure, _, _ = evaluate_against_ground_truth(df, poem_clusters)
    reconstruction_rate, n_perfect, n_total_gt = calculate_perfect_reconstruction_rate(df, poem_clusters)

    results.append({
        'threshold': thresh,
        'n_poem_clusters': len(set(poem_clusters.values())),
        'n_edges': len(edges),
        'ari': ari,
        'v_measure': v_measure,
        'perfect_reconstruction_rate': reconstruction_rate,
        'n_perfect_clusters': n_perfect,
        'n_total_gt_clusters': n_total_gt
    })

results_df = pd.DataFrame(results)
df.to_csv('dbbe_semantic_results/dbbe_poems_semantic.csv')
results_df.to_csv('dbbe_semantic_results/dbbe_poems_semantic_stats.csv')

results_df = results_df.sort_values('threshold')

plt.figure(figsize=(6,4))
plt.plot(results_df['threshold'], results_df['ari'], marker='o', linestyle='-')
plt.xticks(results_df['threshold'])
plt.xlabel("Jaccard Similarity Threshold")
plt.ylabel("Adjusted Rand Index (ARI)")
plt.title("ARI of Poem Clustering vs Threshold")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.savefig("dbbe_semantic_results/ari_poemlevel_sem_dbbe.png", dpi=300)
plt.close()

df = pd.read_csv("dbbe_semantic_results/all_threshold_resolution_results.csv")

df = df[~df['threshold'].isin([0.85, 0.75])]
df = df[df['resolution'] <= 1.0]

df['resolution'] = df['resolution'].round(4)
df['threshold'] = df['threshold'].round(4)
df['ari'] = df['ari'].round(4)

heatmap_data = df.pivot(index='resolution', columns='threshold', values='ari')

plt.figure(figsize=(8, 6))
sns.heatmap(
    heatmap_data, 
    annot=True, 
    fmt=".4f", 
    cmap="viridis", 
    cbar_kws={'label': 'ARI'},
    annot_kws={"fontsize":14}
)
plt.ylabel("Resolution")
plt.xlabel("Similarity Threshold")
plt.title("ARI heatmap across Threshold and Resolution", fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()

plt.savefig("dbbe_semantic_results/ari_verselevel_sem_dbbe.png", dpi=300)
plt.close()

print("\nAll outputs saved to dbbe_semantic_results/")

In [ ]:
# Helper printing verses that are orthographically dissimilar but share a cluster
CLEAN_PATTERN = re.compile(r'[^\w\s]')
WHITESPACE_PATTERN = re.compile(r'\s+')

def preprocess_text(text: str, options: Dict[str, bool] = None) -> str:
    if options is None:
        options = {'lowercase': True, 'remove_diacritics': True, 'remove_punctuation': True}
    text = str(text)
    if options.get('lowercase', True):
        text = text.lower()
    if options.get('remove_diacritics', True):
        text = unicodedata.normalize('NFD', text)
        text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')
        text = unicodedata.normalize('NFC', text)
    else:
        text = unicodedata.normalize('NFC', text)
    if options.get('remove_punctuation', True):
        text = CLEAN_PATTERN.sub('', text)
    text = WHITESPACE_PATTERN.sub(' ', text)
    return text.strip()

df = pd.read_csv("dbbe_semantic_results/dbbe_poems_semantic.csv")
poems = (
    df.sort_values(["idoriginal_poem", "order"])
      .groupby("idoriginal_poem")["verse"]
      .apply(list)
      .to_dict()
)

poem_clusters = (
    df.groupby("idoriginal_poem")["poem_cluster_id"]
      .first()
      .to_dict()
)

cluster_to_poems = defaultdict(list)
for pid, cid in poem_clusters.items():
    cluster_to_poems[cid].append(pid)
def versewise_min_distance(v1, v2):
    v1_proc = [preprocess_text(v) for v in v1]
    v2_proc = [preprocess_text(v) for v in v2]
    
    unmatched = set(range(len(v2_proc)))
    total_dist = 0
    
    for va in v1_proc:
        if not unmatched:
            break
        best_idx = min(unmatched, key=lambda i: jaccard_distance(va, v2_proc[i]))
        total_dist += jaccard_distance(va, v2_proc[best_idx])
        unmatched.remove(best_idx)
        
    return total_dist / max(len(v1_proc), len(v2_proc))

def shingles(s, k=3):
    s = preprocess_text(s)
    return {s[i:i+k] for i in range(len(s)-k+1)} if len(s) >= k else {s}

def jaccard_distance(a, b):

    A, B = shingles(a), shingles(b)
    inter = len(A & B)
    union = len(A | B)
    return 1 - (inter / union if union else 0)

results = [] 

MAX_VERSES = 4
MAX_CLUSTER_SIZE = 5  

for cid, poem_ids in cluster_to_poems.items():
    if len(poem_ids) > MAX_CLUSTER_SIZE:
        continue

    if len(poem_ids) < 2:
        continue

    for p1, p2 in combinations(poem_ids, 2):
        v1, v2 = poems[p1], poems[p2]

        verse_count = len(v1)

        if verse_count > MAX_VERSES:
            continue

        dist = versewise_min_distance(v1, v2)

        results.append((cid, p1, p2, dist, verse_count))

results.sort(key=lambda x: x[3], reverse=True)  

filtered_results = [r for r in results if r[4] > 1]

TOP = 20
for cid, p1, p2, dist, vcount in filtered_results[:TOP]:
    print(f"\n=== Cluster {cid} | Poems in cluster <5 | Verses {vcount} | Distance {dist:.3f} ===")

    print(f"\nPoem {p1}:")
    for line in poems[p1]:
        print("  ", line)

    print(f"\nPoem {p2}:")
    for line in poems[p2]:
        print("  ", line)

In [ ]:
# Helper printing all verses that were clustered together with a given target idgroup
df = pd.read_csv("dbbe_semantic_results/dbbe_poems_semantic.csv")

target_idgroup = 831
threshold_cols = ['cluster_t70', 'cluster_t75', 'cluster_t80', 'cluster_t85', 'cluster_t90', 'cluster_leiden_fixed']
poem_to_cluster = df.groupby('idoriginal_poem')['poem_cluster_id'].first().to_dict()


verse_row = df[df['idgroup'] == target_idgroup]
if verse_row.empty:
    print("idgroup not found")
else:
    print(f"Verse {target_idgroup}: {verse_row['verse'].iloc[0]}\n")dbbe_semantic_results

    for col in threshold_cols:
        cluster_id = verse_row[col].iloc[0]
        same_cluster = df[df[col] == cluster_id]
        same_cluster_sorted = same_cluster.sort_values(['idoriginal_poem', 'order'])

        poem_ids_in_cluster = same_cluster_sorted['idoriginal_poem'].unique().tolist()

        print(f"=== {col} | Cluster {cluster_id} | {len(poem_ids_in_cluster)} poems ===")

        for _, row in same_cluster_sorted.iterrows():
            poem_id = row['idoriginal_poem']
            verse_text = row['verse']
            type_id = row['type_id']
            verse_group = row['idgroup']

            poem_cluster = poem_to_cluster.get(poem_id, "N/A")
            print(f"  Poem ID: {poem_id} | Verse: {verse_text} | Poem Cluster: {poem_cluster} | Type ID: {type_id} | Verse Group {verse_group}")

        print("\n")


In [ ]:
# Helper printing all poems for a poem_cluster_id
df = pd.read_csv("dbbe_semantic_results/dbbe_poems_semantic.csv")
target_cluster_id = 36240

cluster_df = df[df['poem_cluster_id'] == target_cluster_id]

poems_in_cluster = (
    cluster_df.sort_values(['idoriginal_poem', 'order'])
              .groupby('idoriginal_poem')['verse']
              .apply(list)
              .to_dict()
)

for poem_id, verses in poems_in_cluster.items():
    print(f"\nPoem {poem_id}:")
    for line in verses:
        print("  ", line)

# 2. Full dataset

## 2.1 Verse level clustering

In [ ]:
import re
import unicodedata
from typing import Dict, List, Tuple
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from collections import defaultdict, Counter
import time
import pickle
import gzip
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import igraph as ig
import leidenalg as la
import logging
from concurrent.futures import ProcessPoolExecutor, as_completed
import faiss
from functools import partial
import os
import multiprocessing as mp

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger(__name__)

# ============================================================================
# CONFIGURATION
# ============================================================================
CHECKPOINT_DIR = Path("/scratch/gent/vo/000/gvo00042/vsc48660/full_semantic_clustering_checkpoints")
RESULTS_DIR = Path("full_semantic_results_test")
CHECKPOINT_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(exist_ok=True)

# Optimized parameters
N_NEIGHBORS = 100
BATCH_SIZE = 10000
N_THREADS = 16
FAISS_NPROBE = 16

# SPEED OPTIMIZATIONS
SAMPLE_SIZE = 15000      # Sample for parameter selection
N_BOOTSTRAP = 2          # Reduced from 3 (still captures stability)
STABILITY_PAIRS = 2000   # Reduced from 3000 (still accurate)
MAX_WORKERS = min(16, mp.cpu_count())  # Parallel workers

logger.info("="*80)
logger.info("FAST JOINT THRESHOLD-RESOLUTION PARAMETER SWEEP")
logger.info(f"Sample: {SAMPLE_SIZE}, Bootstrap: {N_BOOTSTRAP}, Workers: {MAX_WORKERS}")
logger.info("="*80)

# ============================================================================
# PREPROCESSING UTILITIES
# ============================================================================
CLEAN_PATTERN = re.compile(r'[^\w\s]')
WHITESPACE_PATTERN = re.compile(r'\s+')

def preprocess_text(text: str, options: Dict[str, bool] = None) -> str:
    if options is None:
        options = {'lowercase': True, 'remove_diacritics': True, 'remove_punctuation': True}
    text = str(text)
    if options.get('lowercase', True):
        text = text.lower()
    if options.get('remove_diacritics', True):
        text = unicodedata.normalize('NFD', text)
        text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')
        text = unicodedata.normalize('NFC', text)
    else:
        text = unicodedata.normalize('NFC', text)
    if options.get('remove_punctuation', True):
        text = CLEAN_PATTERN.sub('', text)
    text = WHITESPACE_PATTERN.sub(' ', text)
    return text.strip()

def cls_pooling(model_output):
    return model_output[0][:, 0]

# ============================================================================
# LOAD OR CREATE CHECKPOINT
# ============================================================================
embeddings_file = CHECKPOINT_DIR / 'embeddings.npz'
metadata_file = CHECKPOINT_DIR / 'metadata.pkl.gz'

if embeddings_file.exists() and metadata_file.exists():
    logger.info("\n" + "="*80)
    logger.info("📂 LOADING CHECKPOINT")
    logger.info("="*80)
    
    start_time = time.time()
    
    embeddings_data = np.load(embeddings_file)
    embeddings = embeddings_data['embeddings'].astype('float32')
    
    with gzip.open(metadata_file, 'rb') as f:
        metadata = pickle.load(f)
    
    df = pd.read_parquet(CHECKPOINT_DIR / 'df_minimal.parquet')
    source_datasets = metadata['source_datasets']
    dataset_to_indices = metadata['dataset_to_indices']
    
    logger.info(f"✓ Loaded {len(embeddings):,} embeddings in {time.time()-start_time:.1f}s")
    
    LOADED_FROM_CHECKPOINT = True

else:
    logger.info("\n" + "="*80)
    logger.info("NO CHECKPOINT FOUND - CREATING EMBEDDINGS")
    logger.info("="*80)
    
    csv_path = 'concatenated.csv'
    df = pd.read_csv(csv_path)
    df = df.dropna(subset=['verse', 'source_dataset'])
    df = df[df['verse'].fillna('').astype(str).str.len() >= 20]
    df = df[df['verse'].fillna('').astype(str).str.len() < 256]
    df['verse'] = df['verse'].apply(preprocess_text)
    df = df.reset_index(drop=True)
    
    logger.info(f"Total verses: {len(df):,}")
    
    from transformers import AutoTokenizer, AutoModel
    
    model_name = 'kevinkrahn/shlm-grc-en'
    logger.info(f"Loading model: {model_name}")
    model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logger.info(f"Using device: {device}")
    model = model.to(device)
    model.eval()
    
    logger.info("Computing embeddings...")
    verses = df['verse'].tolist()
    embeddings = []
    batch_size = 32
    
    for i in tqdm(range(0, len(verses), batch_size), desc="Embedding"):
        try:
            batch = verses[i:i+batch_size]
            encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
            encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
            
            with torch.no_grad():
                model_output = model(**encoded_input)
            
            batch_embeddings = cls_pooling(model_output).cpu().numpy()
            embeddings.append(batch_embeddings)
        except Exception as e:
            logger.warning(f"Error at batch {i}: {e}")
            embeddings.append(np.zeros((len(batch), model.config.hidden_size), dtype=np.float32))
    
    embeddings = np.vstack(embeddings).astype('float32')
    embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    
    source_datasets = df['source_dataset'].values
    dataset_to_indices = defaultdict(list)
    for idx, dataset in enumerate(source_datasets):
        dataset_to_indices[dataset].append(idx)
    
    logger.info("\n💾 Saving checkpoint...")
    
    np.savez_compressed(CHECKPOINT_DIR / 'embeddings.npz', embeddings=embeddings)
    
    essential_cols = ['verse', 'source_dataset']
    for col in ['idoriginal_poem', 'idgroup']:
        if col in df.columns:
            essential_cols.append(col)
    
    df_minimal = df[essential_cols].copy()
    for col in df_minimal.columns:
        if df_minimal[col].dtype == 'object':
            df_minimal[col] = df_minimal[col].astype(str)
            df_minimal[col] = df_minimal[col].replace('nan', None).replace('None', None)
    
    df_minimal.to_parquet(CHECKPOINT_DIR / 'df_minimal.parquet', 
                          compression='gzip', index=True)
    
    metadata = {
        'source_datasets': source_datasets,
        'dataset_to_indices': dataset_to_indices
    }
    with gzip.open(CHECKPOINT_DIR / 'metadata.pkl.gz', 'wb') as f:
        pickle.dump(metadata, f, protocol=pickle.HIGHEST_PROTOCOL)
    
    logger.info("✓ Checkpoint saved")
    LOADED_FROM_CHECKPOINT = False

# ============================================================================
# FAST SAMPLING AND INDEXING
# ============================================================================
logger.info("\n" + "="*80)
logger.info("FAST SAMPLE PREPARATION")
logger.info("="*80)

def stratified_sample(df, n_sample=15000):
    """Stratified sampling across datasets"""
    datasets = df['source_dataset'].unique()
    total_size = len(df)
    sample_indices = []
    
    for dataset in datasets:
        dataset_indices = df[df['source_dataset'] == dataset].index.tolist()
        dataset_size = len(dataset_indices)
        proportion = dataset_size / total_size
        n_from_dataset = int(n_sample * proportion)
        n_from_dataset = min(n_from_dataset, dataset_size)
        if n_from_dataset > 0:
            sampled = np.random.choice(dataset_indices, size=n_from_dataset, replace=False)
            sample_indices.extend(sampled)
    
    return np.array(sorted(sample_indices))

# Sample
sample_indices = stratified_sample(df, n_sample=SAMPLE_SIZE)
sample_embeddings = embeddings[sample_indices].copy()
sample_dataset_map = np.array([source_datasets[i] for i in sample_indices])

logger.info(f"Sample size: {len(sample_indices):,}")

# Build FAISS index
logger.info("Building FAISS index...")
start_time = time.time()
dimension = embeddings.shape[1]
index_sample = faiss.IndexFlatIP(dimension)
faiss.normalize_L2(sample_embeddings)
index_sample.add(sample_embeddings)

k = min(200, len(sample_embeddings) - 1)
similarities, indices = index_sample.search(sample_embeddings, k)

logger.info(f"✓ Neighbor search complete in {time.time()-start_time:.1f}s")

# ============================================================================
# OPTIMIZED PARAMETER GRID (COARSE-TO-FINE)
# ============================================================================

# STAGE 1: Coarse grid for initial exploration
threshold_percentiles_coarse = [96, 97, 98,99]   # 4 values
resolutions_coarse = np.logspace(-6, -1, 8)      # 8 values spanning the range

# STAGE 2: Fine grid around best coarse result (done after coarse sweep)

logger.info(f"\nCoarse parameter grid:")
logger.info(f"  Thresholds: {threshold_percentiles_coarse}")
logger.info(f"  Resolutions: {len(resolutions_coarse)} values")
logger.info(f"  Total: {len(threshold_percentiles_coarse) * len(resolutions_coarse)} combinations")

# ============================================================================
# PRECOMPUTE CROSS-DATASET SIMILARITIES AND THRESHOLDS
# ============================================================================
logger.info("\nPrecomputing cross-dataset similarities...")
start_time = time.time()

# Vectorized approach - much faster
cross_similarities = []
for i in range(len(sample_embeddings)):
    # Boolean mask for cross-dataset neighbors
    neighbor_datasets = sample_dataset_map[indices[i, 1:]]  # Skip self
    cross_mask = neighbor_datasets != sample_dataset_map[i]
    cross_similarities.extend(similarities[i, 1:][cross_mask])

cross_similarities = np.array(cross_similarities)
logger.info(f"✓ Collected {len(cross_similarities):,} pairs in {time.time()-start_time:.1f}s")

# Precompute all threshold values
all_percentiles = list(range(50, 100, 5))  # For potential fine-tuning later
threshold_lookup = {p: np.percentile(cross_similarities, p) for p in all_percentiles}

logger.info(f"Threshold range: {threshold_lookup[50]:.4f} (P50) to {threshold_lookup[95]:.4f} (P95)")

# ============================================================================
# OPTIMIZED EVALUATION FUNCTIONS
# ============================================================================

def compute_stability_fast(partitions_list, n_nodes, sample_size=STABILITY_PAIRS):
    """Fast stability computation with smaller sample"""
    if n_nodes < 100:
        return 0.0
    
    n_partitions = len(partitions_list)
    
    # Sample fewer pairs
    n_sample_pairs = min(sample_size, n_nodes * (n_nodes - 1) // 2)
    
    # Vectorized pair sampling
    pairs_i = np.random.randint(0, n_nodes, n_sample_pairs)
    pairs_j = np.random.randint(0, n_nodes, n_sample_pairs)
    valid = pairs_i != pairs_j
    pairs_i = pairs_i[valid][:n_sample_pairs]
    pairs_j = pairs_j[valid][:n_sample_pairs]
    
    # Vectorized co-clustering check
    coclustering = 0
    for membership in partitions_list:
        membership_arr = np.array(membership)
        matches = membership_arr[pairs_i] == membership_arr[pairs_j]
        coclustering += np.sum(matches)
    
    stability = coclustering / (len(pairs_i) * n_partitions)
    return stability

def evaluate_single_combination(args):
    """
    Evaluate a single (threshold, resolution) combination.
    This function is designed to be called in parallel.
    """
    threshold_pct, threshold, resolution, edges_data, weights_data, dataset_map, n_nodes = args
    
    try:
        # Filter edges for this threshold
        edge_mask = weights_data >= threshold
        edges = edges_data[edge_mask]
        weights = weights_data[edge_mask]
        
        if len(edges) == 0:
            return None
        
        # Build graph
        g = ig.Graph(n=n_nodes, edges=edges.tolist(), directed=False)
        g.es['weight'] = weights.tolist()
        
        # Bootstrap clustering (reduced iterations)
        bootstrap_memberships = []
        bootstrap_qualities = []
        
        for seed in range(N_BOOTSTRAP):
            partition = la.find_partition(
                g,
                la.CPMVertexPartition,
                weights='weight',
                resolution_parameter=resolution,
                n_iterations=5,  # Reduced from 10
                seed=seed
            )
            bootstrap_memberships.append(partition.membership)
            bootstrap_qualities.append(partition.quality())
        
        # Analyze
        membership = bootstrap_memberships[0]
        n_clusters = len(set(membership))
        
        if n_clusters == 0 or n_clusters == n_nodes:
            return None
        
        # Compute metrics
        stability = compute_stability_fast(bootstrap_memberships, n_nodes)
        avg_size = n_nodes / n_clusters
        
        # Cross-dataset metrics
        cluster_datasets = defaultdict(set)
        cluster_sizes = defaultdict(int)
        
        for idx, cid in enumerate(membership):
            cluster_datasets[cid].add(dataset_map[idx])
            cluster_sizes[cid] += 1
        
        n_cross_clusters = sum(1 for datasets in cluster_datasets.values() 
                              if len(datasets) > 1)
        pct_cross_clusters = (n_cross_clusters / n_clusters * 100) if n_clusters > 0 else 0
        n_singleton = sum(1 for size in cluster_sizes.values() if size == 1)
        
        avg_quality = np.mean(bootstrap_qualities)
        modularity = g.modularity(membership, weights='weight')
        
        return {
            'threshold_percentile': threshold_pct,
            'threshold_value': threshold,
            'resolution': resolution,
            'n_edges': g.ecount(),
            'graph_density': 2*g.ecount()/(n_nodes*(n_nodes-1)) if n_nodes > 1 else 0,
            'stability': stability,
            'n_clusters': n_clusters,
            'n_singleton': n_singleton,
            'avg_cluster_size': avg_size,
            'n_cross_clusters': n_cross_clusters,
            'pct_cross_clusters': pct_cross_clusters,
            'avg_quality': avg_quality,
            'modularity': modularity
        }
    
    except Exception as e:
        logger.warning(f"Error at P{threshold_pct}, res={resolution:.2e}: {e}")
        return None

# ============================================================================
# PRECOMPUTE EDGE STRUCTURES (MAJOR SPEEDUP)
# ============================================================================
logger.info("\nPrecomputing edge structures...")
start_time = time.time()

# Build complete edge list with weights (only cross-dataset)
all_edges_list = []
all_weights_list = []

for i in range(len(sample_embeddings)):
    dataset_i = sample_dataset_map[i]
    neighbors = indices[i, 1:]  # Skip self
    sims = similarities[i, 1:]
    
    # Vectorized filtering
    valid_mask = neighbors > i  # Avoid duplicates
    valid_neighbors = neighbors[valid_mask]
    valid_sims = sims[valid_mask]
    
    if len(valid_neighbors) > 0:
        neighbor_datasets = sample_dataset_map[valid_neighbors]
        cross_mask = neighbor_datasets != dataset_i
        
        final_neighbors = valid_neighbors[cross_mask]
        final_sims = valid_sims[cross_mask]
        
        for j, sim in zip(final_neighbors, final_sims):
            all_edges_list.append([i, j])
            all_weights_list.append(sim)

# Convert to numpy arrays (shared across all parameter combinations)
all_edges = np.array(all_edges_list, dtype=np.int32)
all_weights = np.array(all_weights_list, dtype=np.float32)

logger.info(f"✓ Precomputed {len(all_edges):,} edges in {time.time()-start_time:.1f}s")
# ============================================================================
# STAGE 1: COARSE PARALLEL SWEEP
# ============================================================================
logger.info("\n" + "="*80)
logger.info("STAGE 1: COARSE PARALLEL SWEEP")
logger.info("="*80)

coarse_args = []
for threshold_pct in threshold_percentiles_coarse:
    # Get threshold value, compute if not in lookup
    if threshold_pct in threshold_lookup:
        threshold = threshold_lookup[threshold_pct]
    else:
        threshold = np.percentile(cross_similarities, threshold_pct)
        threshold_lookup[threshold_pct] = threshold
    
    for resolution in resolutions_coarse:
        coarse_args.append((
            threshold_pct,
            threshold,
            resolution,
            all_edges,
            all_weights,
            sample_dataset_map,
            len(sample_embeddings)
        ))


logger.info(f"Testing {len(coarse_args)} combinations with {MAX_WORKERS} workers...")
start_time = time.time()

# Parallel execution
coarse_results = []
with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = {executor.submit(evaluate_single_combination, args): args 
               for args in coarse_args}
    
    with tqdm(total=len(futures), desc="Coarse sweep") as pbar:
        for future in as_completed(futures):
            result = future.result()
            if result is not None:
                coarse_results.append(result)
            pbar.update(1)

logger.info(f"✓ Coarse sweep complete in {time.time()-start_time:.1f}s")
logger.info(f"  Valid results: {len(coarse_results)} / {len(coarse_args)}")

# ============================================================================
# STAGE 2: FINE SWEEP AROUND BEST COARSE RESULT
# ============================================================================
logger.info("\n" + "="*80)
logger.info("STAGE 2: FINE SWEEP AROUND BEST REGION")
logger.info("="*80)

if len(coarse_results) == 0:
    logger.error("No valid coarse results! Cannot proceed with fine sweep.")
    exit(1)

# Convert to dataframe for analysis
coarse_df = pd.DataFrame(coarse_results)

# SELECTION CRITERION: STABILITY ONLY
# Find best coarse result based on stability
best_coarse = coarse_df.loc[coarse_df['stability'].idxmax()]
best_thresh_pct = best_coarse['threshold_percentile']
best_res_coarse = best_coarse['resolution']

logger.info(f"Best coarse result (by stability): P{best_thresh_pct}, res={best_res_coarse:.2e}")
logger.info(f"  Stability: {best_coarse['stability']:.3f}")
logger.info(f"  Clusters: {best_coarse['n_clusters']:,.0f}")
logger.info(f"  Cross-dataset: {best_coarse['pct_cross_clusters']:.1f}%")

# Define fine grid around best region
# Threshold: ±10 percentile points
thresh_fine = [max(50, best_thresh_pct - 10), 
               max(50, best_thresh_pct - 5),
               best_thresh_pct,
               min(95, best_thresh_pct + 5),
               min(95, best_thresh_pct + 10)]
thresh_fine = sorted(list(set(thresh_fine)))  # Remove duplicates

# Resolution: ±1 order of magnitude (log scale)
log_res = np.log10(best_res_coarse)
res_fine = np.logspace(log_res - 0.5, log_res + 0.5, 7)

logger.info(f"\nFine grid:")
logger.info(f"  Thresholds: {thresh_fine}")
logger.info(f"  Resolutions: {len(res_fine)} values around {best_res_coarse:.2e}")
logger.info(f"  Total: {len(thresh_fine) * len(res_fine)} combinations")

# Prepare fine sweep arguments
fine_args = []
for threshold_pct in thresh_fine:
    if threshold_pct in threshold_lookup:
        threshold = threshold_lookup[threshold_pct]
    else:
        threshold = np.percentile(cross_similarities, threshold_pct)
        threshold_lookup[threshold_pct] = threshold
    
    for resolution in res_fine:
        # Skip if already tested in coarse sweep
        already_tested = any(
            np.isclose(r['threshold_percentile'], threshold_pct) and 
            np.isclose(r['resolution'], resolution, rtol=0.1)
            for r in coarse_results
        )
        if not already_tested:
            fine_args.append((
                threshold_pct,
                threshold,
                resolution,
                all_edges,
                all_weights,
                sample_dataset_map,
                len(sample_embeddings)
            ))

logger.info(f"Testing {len(fine_args)} new combinations...")
start_time = time.time()

# Parallel execution
fine_results = []
with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = {executor.submit(evaluate_single_combination, args): args 
               for args in fine_args}
    
    with tqdm(total=len(futures), desc="Fine sweep") as pbar:
        for future in as_completed(futures):
            result = future.result()
            if result is not None:
                fine_results.append(result)
            pbar.update(1)

logger.info(f"✓ Fine sweep complete in {time.time()-start_time:.1f}s")
logger.info(f"  Valid results: {len(fine_results)} / {len(fine_args)}")

# ============================================================================
# COMBINE AND ANALYZE ALL RESULTS
# ============================================================================
logger.info("\n" + "="*80)
logger.info("FINAL ANALYSIS - SELECTION BY STABILITY")
logger.info("="*80)

# Combine all results
all_results = coarse_results + fine_results
sweep_df = pd.DataFrame(all_results)

# SELECTION CRITERION: STABILITY ONLY
# Sort by stability (highest first)
sweep_df = sweep_df.sort_values('stability', ascending=False)
sweep_df.to_csv(RESULTS_DIR / 'joint_parameter_sweep_results.csv', index=False)

# Find overall best (highest stability)
best_params = sweep_df.iloc[0]
best_threshold = best_params['threshold_value']
best_resolution = best_params['resolution']

logger.info("\n" + "="*80)
logger.info("TOP 5 PARAMETER COMBINATIONS (BY STABILITY)")
logger.info("="*80)

for idx, (i, row) in enumerate(sweep_df.head(10).iterrows(), 1):
    logger.info(f"\n#{idx}. Threshold: P{row['threshold_percentile']} ({row['threshold_value']:.4f}), "
               f"Resolution: {row['resolution']:.2e}")
    logger.info(f"     Stability: {row['stability']:.3f}")
    logger.info(f"     Clusters: {row['n_clusters']:,.0f}, Singletons: {row['n_singleton']:,.0f}")
    logger.info(f"     Cross-dataset: {row['n_cross_clusters']:,.0f} ({row['pct_cross_clusters']:.1f}%)")
    logger.info(f"     Modularity: {row['modularity']:.3f}, Quality: {row['avg_quality']:.3f}")

logger.info("\n" + "="*80)
logger.info("🎯 SELECTED PARAMETERS (HIGHEST STABILITY)")
logger.info("="*80)
logger.info(f"Threshold: P{best_params['threshold_percentile']} = {best_threshold:.4f}")
logger.info(f"Resolution: {best_resolution:.6e}")
logger.info(f"Stability: {best_params['stability']:.3f}")
logger.info(f"Clusters: {best_params['n_clusters']:,.0f}")
logger.info(f"Cross-dataset: {best_params['pct_cross_clusters']:.1f}%")
logger.info("="*80)

# ============================================================================
# QUICK VISUALIZATION
# ============================================================================
logger.info("\nCreating visualization...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Heatmap of stability scores
pivot_data = sweep_df.pivot_table(
    values='stability',
    index='resolution',
    columns='threshold_percentile',
    aggfunc='first'
)

# Round resolution values to 3 decimal places for display
pivot_data.index = [f"{res:.3e}" for res in pivot_data.index]

ax = axes[0, 0]
sns.heatmap(pivot_data, annot=True, fmt='.3f', cmap='RdYlGn', ax=ax, 
           cbar_kws={'label': 'Stability'})
ax.set_ylabel('Resolution', fontweight='bold')
ax.set_xlabel('Threshold Percentile', fontweight='bold')
ax.set_title('Stability Heatmap', fontweight='bold', fontsize=14)

# 2. Stability distribution
ax = axes[0, 1]
ax.hist(sweep_df['stability'], bins=30, color='#0173B2', alpha=0.7, edgecolor='black')
ax.axvline(best_params['stability'], color='red', linestyle='--', linewidth=2, 
          label=f"Best: {best_params['stability']:.3f}")
ax.set_xlabel('Stability', fontweight='bold')
ax.set_ylabel('Frequency', fontweight='bold')
ax.set_title('Stability Distribution', fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# 3. Stability vs resolution for different thresholds
ax = axes[1, 0]
for thresh_pct in sorted(sweep_df['threshold_percentile'].unique()):
    data = sweep_df[sweep_df['threshold_percentile'] == thresh_pct]
    ax.plot(data['resolution'], data['stability'], 'o-', 
           label=f'P{thresh_pct}', alpha=0.7, markersize=4)
ax.axhline(best_params['stability'], color='red', linestyle='--', 
          linewidth=1, alpha=0.5, label=f'Best: {best_params["stability"]:.3f}')
ax.set_xlabel('Resolution', fontweight='bold')
ax.set_ylabel('Stability', fontweight='bold')
ax.set_title('Stability vs Resolution', fontweight='bold')
ax.set_xscale('log')
ax.legend(fontsize=8)
ax.grid(alpha=0.3)

# 4. Stability vs clusters (colored by threshold)
ax = axes[1, 1]
scatter = ax.scatter(sweep_df['n_clusters'], sweep_df['stability'],
                    c=sweep_df['threshold_value'], cmap='viridis',
                    s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
ax.scatter(best_params['n_clusters'], best_params['stability'],
          color='red', s=200, marker='*', edgecolors='black', linewidth=2,
          label='Best', zorder=10)
ax.set_xlabel('Number of Clusters', fontweight='bold')
ax.set_ylabel('Stability', fontweight='bold')
ax.set_title('Stability vs Cluster Count (colored by threshold)', fontweight='bold')
ax.set_xscale('log')
plt.colorbar(scatter, ax=ax, label='Threshold Value')
ax.legend()
ax.grid(alpha=0.3)

fig.suptitle(f'Fast Joint Parameter Sweep - Stability-Based Selection (n={len(sweep_df)} combinations)', 
            fontsize=16, fontweight='bold', y=0.995)

plt.tight_layout()
plot_path = RESULTS_DIR / 'fast_parameter_sweep_summary.png'
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
logger.info(f"✓ Plot saved: {plot_path}")
plt.close()


# ============================================================================
# SAVE RESULTS
# ============================================================================

summary = {
    'best_threshold_percentile': best_params['threshold_percentile'],
    'best_threshold_value': best_threshold,
    'best_resolution': best_resolution,
    'stability': best_params['stability'],
    'n_clusters': best_params['n_clusters'],
    'n_singleton': best_params['n_singleton'],
    'avg_cluster_size': best_params['avg_cluster_size'],
    'n_cross_clusters': best_params['n_cross_clusters'],
    'pct_cross_clusters': best_params['pct_cross_clusters'],
    'modularity': best_params['modularity'],
    'n_combinations_tested': len(sweep_df),
    'sample_size': SAMPLE_SIZE,
    'n_bootstrap': N_BOOTSTRAP,
    'selection_criterion': 'stability'
}

pd.DataFrame([summary]).to_csv(RESULTS_DIR / 'best_parameters_summary.csv', index=False)

params_for_clustering = {
    'threshold': best_threshold,
    'resolution': best_resolution
}

with open(RESULTS_DIR / 'optimal_parameters.pkl', 'wb') as f:
    pickle.dump(params_for_clustering, f)

logger.info(f"\n✓ Results saved to: {RESULTS_DIR}")
logger.info("="*80)
logger.info("✅ FAST JOINT PARAMETER SWEEP COMPLETE")
logger.info("="*80)
# ============================================================================
# STAGE 3: FAST GRAPH CONSTRUCTION WITH FAISS
# ============================================================================
logger.info("\n" + "="*80)
logger.info("STAGE 3: FAST APPROXIMATE GRAPH CONSTRUCTION")
logger.info("="*80)

# Normalize embeddings
logger.info("Normalizing embeddings...")
faiss.normalize_L2(embeddings)

# Build quantized FAISS index for speed
logger.info("Building FAISS IVF index...")
start_time = time.time()

# Optimize nlist based on data size (rule of thumb: 4*sqrt(n) to 16*sqrt(n))
nlist = min(int(4 * np.sqrt(len(embeddings))), 8192)  # Cap at 8192 for memory
nprobe = min(FAISS_NPROBE, nlist // 4)  # Don't probe more than 1/4 of clusters

logger.info(f"FAISS params: nlist={nlist}, nprobe={nprobe}")

quantizer = faiss.IndexFlatIP(dimension)
index_full = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_INNER_PRODUCT)

# Try to use GPU if available (massive speedup)
try:
    res = faiss.StandardGpuResources()
    index_full = faiss.index_cpu_to_gpu(res, 0, index_full)
    logger.info("✓ Using GPU acceleration for FAISS")
    on_gpu = True
except Exception as e:
    logger.info(f"GPU not available ({e}), using CPU")
    on_gpu = False

# Train on a sample if data is very large
if len(embeddings) > 1000000:
    train_sample_size = min(500000, len(embeddings))
    train_indices = np.random.choice(len(embeddings), train_sample_size, replace=False)
    train_data = embeddings[train_indices]
    logger.info(f"Training on sample of {train_sample_size:,} vectors...")
    index_full.train(train_data)
else:
    index_full.train(embeddings)

index_full.add(embeddings)
index_full.nprobe = nprobe

logger.info(f"✓ FAISS index built in {time.time()-start_time:.1f}s")

# Search for neighbors in LARGE batches (FAISS is optimized for batch queries)
logger.info(f"Searching for {N_NEIGHBORS} nearest neighbors...")
SEARCH_BATCH_SIZE = 50000  # Much larger batches for FAISS efficiency

start_time = time.time()
all_similarities = []
all_indices = []

n_search_batches = (len(embeddings) + SEARCH_BATCH_SIZE - 1) // SEARCH_BATCH_SIZE
logger.info(f"Processing {n_search_batches} search batches of size {SEARCH_BATCH_SIZE:,}...")

for i in tqdm(range(0, len(embeddings), SEARCH_BATCH_SIZE), desc="Neighbor search", total=n_search_batches):
    batch_end = min(i + SEARCH_BATCH_SIZE, len(embeddings))
    batch_emb = embeddings[i:batch_end]
    
    # FAISS search is MUCH faster with larger batches
    D, I = index_full.search(batch_emb, N_NEIGHBORS)
    all_similarities.append(D)
    all_indices.append(I)

all_similarities = np.vstack(all_similarities)
all_indices = np.vstack(all_indices)

logger.info(f"✓ Neighbor search complete in {time.time()-start_time:.1f}s")

# ============================================================================
# STAGE 4: MEMORY-EFFICIENT EDGE CONSTRUCTION
# ============================================================================
logger.info("\n" + "="*80)
logger.info("STAGE 4: MEMORY-EFFICIENT EDGE CONSTRUCTION")
logger.info("="*80)

# Check if edge checkpoint exists
edge_checkpoint = CHECKPOINT_DIR / 'edges_checkpoint.npz'

if edge_checkpoint.exists():
    logger.info("Found edge checkpoint - loading...")
    edge_data = np.load(edge_checkpoint)
    all_edges = [(int(i), int(j)) for i, j in edge_data['edges']]
    all_weights = edge_data['weights'].tolist()
    logger.info(f"✓ Loaded {len(all_edges):,} edges from checkpoint")
    
else:
    logger.info("Building edge list with cross-dataset filtering...")
    start_time = time.time()
    
    # Strategy: Use numpy vectorization for speed while managing memory
    # We'll process in chunks and use boolean masking
    
    all_edges = []
    all_weights = []
    
    EDGE_BATCH_SIZE = 20000  # Process this many nodes at once
    
    for batch_start in tqdm(range(0, len(embeddings), EDGE_BATCH_SIZE), desc="Building edges"):
        batch_end = min(batch_start + EDGE_BATCH_SIZE, len(embeddings))
        
        # Get batch of node indices
        batch_size = batch_end - batch_start
        
        # For each node in batch, get its neighbors
        for local_idx in range(batch_size):
            node_idx = batch_start + local_idx
            dataset_i = source_datasets[node_idx]
            
            # Get neighbors (skip self at index 0)
            neighbors = all_indices[node_idx, 1:]
            sims = all_similarities[node_idx, 1:]
            
            # Vectorized filtering
            # 1. Only neighbors with index > node_idx (avoid duplicates)
            # 2. Similarity >= threshold
            # 3. Cross-dataset
            valid_mask = (neighbors > node_idx) & (sims >= threshold)
            valid_neighbors = neighbors[valid_mask]
            valid_sims = sims[valid_mask]
            
            # Check cross-dataset for valid neighbors
            if len(valid_neighbors) > 0:
                neighbor_datasets = np.array([source_datasets[n] for n in valid_neighbors])
                cross_dataset_mask = neighbor_datasets != dataset_i
                
                final_neighbors = valid_neighbors[cross_dataset_mask]
                final_sims = valid_sims[cross_dataset_mask]
                
                # Add edges
                for neighbor, sim in zip(final_neighbors, final_sims):
                    all_edges.append((node_idx, int(neighbor)))
                    all_weights.append(float(sim))
        
        # Periodic memory cleanup
        if (batch_start // EDGE_BATCH_SIZE) % 10 == 0 and batch_start > 0:
            import gc
            gc.collect()
    
    logger.info(f"✓ {len(all_edges):,} cross-dataset edges in {time.time()-start_time:.1f}s")
    
    # Save edge checkpoint (in case clustering crashes)
    logger.info("Saving edge checkpoint...")
    np.savez_compressed(
        CHECKPOINT_DIR / 'edges_checkpoint.npz',
        edges=np.array(all_edges, dtype=np.int32),
        weights=np.array(all_weights, dtype=np.float32)
    )
    logger.info("✓ Edge checkpoint saved")

# ============================================================================
# STAGE 5: HIERARCHICAL LEIDEN CLUSTERING
# ============================================================================
logger.info("\n" + "="*80)
logger.info("STAGE 5: HIERARCHICAL LEIDEN CLUSTERING")
logger.info("="*80)

# Build graph
logger.info("Building graph...")
g = ig.Graph(n=len(embeddings), edges=all_edges, directed=False)
g.es['weight'] = all_weights

logger.info(f"Full graph: {g.vcount():,} nodes, {g.ecount():,} edges")

# Step 1: Coarse clustering
logger.info("\nStep 1: Coarse clustering (fast)...")
start_time = time.time()

coarse_partition = la.find_partition(
    g,
    la.CPMVertexPartition,
    weights='weight',
    resolution_parameter=0.01,  # Very low for speed
    n_iterations=3,
    seed=42
)

coarse_labels = np.array(coarse_partition.membership)
n_coarse = len(set(coarse_labels))

logger.info(f"✓ {n_coarse:,} coarse clusters in {time.time()-start_time:.1f}s")

# Identify cross-dataset clusters
coarse_cluster_info = defaultdict(lambda: {'datasets': set(), 'nodes': []})

for idx, cid in enumerate(coarse_labels):
    coarse_cluster_info[cid]['datasets'].add(source_datasets[idx])
    coarse_cluster_info[cid]['nodes'].append(idx)

cross_dataset_clusters = [cid for cid, info in coarse_cluster_info.items()
                          if len(info['datasets']) > 1]

logger.info(f"Cross-dataset coarse clusters: {len(cross_dataset_clusters):,}")

# Step 2: Refine cross-dataset clusters
logger.info(f"\nStep 2: Refining with resolution {resolution:.4f}...")

final_labels = coarse_labels.copy()
next_cluster_id = n_coarse

# Sort by size
cluster_sizes = [(cid, len(coarse_cluster_info[cid]['nodes'])) 
                 for cid in cross_dataset_clusters]
cluster_sizes.sort(key=lambda x: x[1])

refined_count = 0
skipped_small = 0
skipped_large = 0

for coarse_cid, size in tqdm(cluster_sizes, desc="Refining"):
    if size < 10:
        skipped_small += 1
        continue
    
    if size > 100000:
        skipped_large += 1
        continue
    
    try:
        cluster_nodes = coarse_cluster_info[coarse_cid]['nodes']
        subg = g.subgraph(cluster_nodes)
        
        if subg.ecount() > 0:
            sub_partition = la.find_partition(
                subg,
                la.CPMVertexPartition,
                weights='weight',
                resolution_parameter=resolution,
                n_iterations=10,
                seed=42
            )
            
            sub_labels = np.array(sub_partition.membership)
            
            # Only refine if we split the cluster
            if len(set(sub_labels)) > 1:
                for sub_idx, sub_cid in enumerate(sub_labels):
                    global_idx = cluster_nodes[sub_idx]
                    final_labels[global_idx] = next_cluster_id + sub_cid
                
                next_cluster_id += len(set(sub_labels))
                refined_count += 1
    
    except Exception as e:
        logger.warning(f"Error refining cluster {coarse_cid}: {e}")
        continue

logger.info(f"✓ Refined: {refined_count:,}, Skipped small: {skipped_small:,}, "
            f"Skipped large: {skipped_large:,}")

# ============================================================================
# ANALYZE AND SAVE RESULTS
# ============================================================================
logger.info("\n" + "="*80)
logger.info("ANALYZING RESULTS")
logger.info("="*80)

cluster_datasets = defaultdict(set)
cluster_verses = defaultdict(list)

for idx, cid in enumerate(final_labels):
    cluster_datasets[cid].add(source_datasets[idx])
    cluster_verses[cid].append(idx)

n_total_clusters = len(set(final_labels))
n_cross = sum(1 for ds in cluster_datasets.values() if len(ds) > 1)
cross_verses = sum(len(cluster_verses[cid]) for cid in set(final_labels)
                  if len(cluster_datasets[cid]) > 1)

logger.info(f"\nFINAL RESULTS:")
logger.info(f"  Total clusters: {n_total_clusters:,}")
logger.info(f"  Cross-dataset clusters: {n_cross:,} ({n_cross/n_total_clusters*100:.1f}%)")
logger.info(f"  Cross-dataset verses: {cross_verses:,} ({cross_verses/len(final_labels)*100:.1f}%)")

# Add to dataframe
df['cluster_id'] = final_labels

# Create cluster summary
cluster_info = []
for cid, datasets in cluster_datasets.items():
    if len(datasets) > 1:
        verses = cluster_verses[cid]
        counts = Counter(source_datasets[i] for i in verses)
        cluster_info.append({
            'cluster_id': cid,
            'size': len(verses),
            'n_datasets': len(datasets),
            'datasets': ', '.join(sorted(datasets)),
            'dataset_counts': dict(counts)
        })

cluster_info_df = pd.DataFrame(cluster_info).sort_values('size', ascending=False)

logger.info(f"\nTop 20 cross-dataset clusters:")
if len(cluster_info_df) > 0:
    print(cluster_info_df.head(20).to_string(index=False))

# Save results
logger.info("\n" + "="*80)
logger.info("SAVING RESULTS")
logger.info("="*80)

df.to_csv(RESULTS_DIR / "concatenated_cross_dataset_clusters.csv", index=False)
logger.info(f"✓ Full results: {RESULTS_DIR / 'concatenated_cross_dataset_clusters.csv'}")

cross_mask = df['cluster_id'].map(lambda cid: len(cluster_datasets[cid]) > 1)
df_cross = df[cross_mask].copy()

df_cross = df_cross[['verse', 'source_dataset', 'cluster_id']]  # include other relevant columns if needed
df_cross.to_csv(RESULTS_DIR / "cross_dataset_verses_only.csv", index=False)
logger.info(f"✓ Cross-dataset only: {RESULTS_DIR / 'cross_dataset_verses_only.csv'} ({len(df_cross):,} verses)")


cluster_info_df.to_csv(RESULTS_DIR / 'cross_dataset_clusters_summary.csv', index=False)
logger.info(f"✓ Cluster summary: {RESULTS_DIR / 'cross_dataset_clusters_summary.csv'}")

# Save final summary
final_summary = {
    'n_verses': len(df),
    'n_datasets': len(set(source_datasets)),
    'threshold': threshold,
    'resolution': resolution,
    'n_total_clusters': n_total_clusters,
    'n_cross_clusters': n_cross,
    'pct_cross_clusters': n_cross/n_total_clusters*100,
    'n_cross_verses': cross_verses,
    'pct_cross_verses': cross_verses/len(df)*100
}


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load sweep results
sweep_df = pd.read_csv("full_semantic_results/joint_parameter_sweep_results.csv")

# Create a new column for x-axis labels: "threshold (PXX)"
sweep_df['threshold_label'] = sweep_df.apply(
    lambda row: f"{row['threshold_value']:.4f} (P{int(row['threshold_percentile'])})", axis=1
)

# Pivot for heatmap: use the new label column
pivot_data = sweep_df.pivot_table(
    values='stability',
    index='resolution',
    columns='threshold_label',
    aggfunc='first'
)

# Round resolution values for display
pivot_data.index = [f"{res:.3e}" for res in pivot_data.index]

# Plot heatmap with viridis colormap
plt.figure(figsize=(14, 8))
sns.heatmap(pivot_data, annot=True, fmt=".3f", cmap='viridis', cbar_kws={'label': 'Stability'})
plt.xlabel("Cosine Similarity Threshold (Percentile)", fontweight='bold')
plt.ylabel("Resolution", fontweight='bold')
plt.title("Stability Heatmap with Threshold Values", fontweight='bold', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('full_semantic_results/threshold_selection_sem_quant.png', dpi=300, bbox_inches='tight')


pd.DataFrame([final_summary]).to_csv(RESULTS_DIR / 'clustering_summary.csv', index=False)
logger.info(f"✓ Summary: {RESULTS_DIR / 'clustering_summary.csv'}")

logger.info("\n" + "="*80)
logger.info("✅ CLUSTERING COMPLETE")
logger.info("="*80)
logger.info(f"All results saved to: {RESULTS_DIR}/")

09:33:36 | INFO | ================================================================================
09:33:36 | INFO | FAST JOINT THRESHOLD-RESOLUTION PARAMETER SWEEP
09:33:36 | INFO | Sample: 15000, Bootstrap: 2, Workers: 16
09:33:36 | INFO | ================================================================================
09:33:36 | INFO | 
09:33:36 | INFO | 📂 LOADING CHECKPOINT
09:33:36 | INFO | ================================================================================
09:34:03 | INFO | ✓ Loaded 1,536,616 embeddings in 26.8s
09:34:03 | INFO | 
09:34:03 | INFO | FAST SAMPLE PREPARATION
09:34:03 | INFO | ================================================================================
09:34:03 | INFO | Sample size: 14,998
09:34:03 | INFO | Building FAISS index...
09:34:04 | INFO | ✓ Neighbor search complete in 0.9s
09:34:04 | INFO | 
Coarse parameter grid:
09:34:04 | INFO |   Thresholds: [96, 97, 98, 99]
09:34:04 | INFO |   Resolutions: 8 values
09:34:04 | INFO |   Total: 32 combina

 cluster_id  size  n_datasets                 datasets                                        dataset_counts
          0  1464           4 dbbe, papyri, phi, rhoby  {'rhoby': 5, 'dbbe': 157, 'phi': 842, 'papyri': 460}
          1  1348           4 dbbe, papyri, phi, rhoby  {'rhoby': 14, 'dbbe': 96, 'phi': 641, 'papyri': 597}
          2  1295           4 dbbe, papyri, phi, rhoby  {'rhoby': 14, 'dbbe': 97, 'phi': 778, 'papyri': 406}
          3  1279           3        dbbe, papyri, phi               {'dbbe': 18, 'phi': 884, 'papyri': 377}
          4  1228           4 dbbe, papyri, phi, rhoby   {'rhoby': 2, 'dbbe': 16, 'phi': 791, 'papyri': 419}
          5  1196           2              papyri, phi                           {'phi': 821, 'papyri': 375}
          6  1124           4 dbbe, papyri, phi, rhoby   {'rhoby': 1, 'dbbe': 14, 'phi': 692, 'papyri': 417}
          7  1114           3        dbbe, papyri, phi                {'dbbe': 6, 'phi': 891, 'papyri': 217}
          8  1101  

09:45:06 | INFO | ✓ Full results: full_semantic_results_test/concatenated_cross_dataset_clusters.csv


In [ ]:
import pandas as pd
from itertools import combinations
from collections import defaultdict

RESULTS_DIR = "full_semantic_results_test"

# Load datasets
df_cross = pd.read_csv(f"{RESULTS_DIR}/concatenated_cross_dataset_clusters.csv")
cluster_summary = pd.read_csv(f"{RESULTS_DIR}/cross_dataset_clusters_summary.csv")

# Only keep clusters with >1 dataset
diverse_clusters = cluster_summary[cluster_summary['n_datasets'] > 1] \
    .sort_values(by=['n_datasets', 'size'], ascending=[False, True]) \
    .head(50)

# Ensure poem IDs are strings
df_cross['idoriginal_poem'] = df_cross['idoriginal_poem'].astype(str)

# Precompute poem_key
df_cross['poem_key'] = list(zip(df_cross['source_dataset'], df_cross['idoriginal_poem']))

# Precompute mapping from poem_key -> set of clusters
poem_to_clusters = df_cross.groupby('poem_key')['cluster_id'].apply(set).to_dict()

# Precompute cluster -> list of unique poems in that cluster
cluster_to_poems = df_cross.groupby('cluster_id')['poem_key'].apply(set).to_dict()

# Iterate over clusters
for cid, cluster_row in diverse_clusters.iterrows():
    if cid not in cluster_to_poems:
        continue

    poems_in_cluster = list(cluster_to_poems[cid])
    
    shared_cluster_results = []

    # Check all cross-dataset poem pairs for shared clusters
    for p1, p2 in combinations(poems_in_cluster, 2):
        if p1[0] == p2[0]:  # skip same dataset
            continue

        shared = (poem_to_clusters[p1] & poem_to_clusters[p2]) - {cid}
        if shared:
            shared_cluster_results.append((p1, p2, shared))

    # Skip cluster if no cross-dataset pairs share other clusters
    if not shared_cluster_results:
        continue

    # Retrieve verses for this cluster
    cluster_verses = df_cross[df_cross['cluster_id'] == cid]

    print(f"\n=== Cluster {cid} ({len(cluster_verses)} verses, {cluster_row['n_datasets']} datasets) ===")

    for _, verse_row in cluster_verses.iterrows():
        dataset = verse_row['source_dataset']
        verse_text = verse_row['verse']
        poem_id = verse_row['idoriginal_poem']
        print(f"[{dataset}] : {poem_id}: {verse_text}")

    print("\n--- Other clusters shared by cross-dataset poem pairs in this cluster ---")
    for p1, p2, shared_clusters in shared_cluster_results:
        print(f"Poem pair {p1} & {p2} co-occur in clusters: {list(shared_clusters)}")


In [1]:


### This prints verses that were clustered together, prioritizing cross-dataset clusters, aiming for verses belonging to small poems, 
### Only showing verses of which the corresponding poem also shares other verses 
import pandas as pd
from itertools import combinations

RESULTS_DIR = "full_semantic_results_test"

# Load datasets
df_cross = pd.read_csv(f"{RESULTS_DIR}/concatenated_cross_dataset_clusters.csv")
cluster_summary = pd.read_csv(f"{RESULTS_DIR}/cross_dataset_clusters_summary.csv")

# Only keep clusters with >1 dataset
diverse_clusters = cluster_summary[cluster_summary['n_datasets'] > 1] \
    .sort_values(by=['n_datasets', 'size'], ascending=[False, True]) \
    .head(50)

# Ensure poem IDs are strings
df_cross['idoriginal_poem'] = df_cross['idoriginal_poem'].astype(str)

# Precompute poem_key
df_cross['poem_key'] = list(zip(df_cross['source_dataset'], df_cross['idoriginal_poem']))

# Compute poem lengths
poem_lengths = df_cross.groupby('poem_key').size().to_dict()

# Precompute mapping from poem_key -> set of clusters (only poems with <10 verses)
poem_to_clusters = {
    k: set(v['cluster_id'])
    for k, v in df_cross.groupby('poem_key')
    if poem_lengths[k] < 10
}

# Precompute cluster -> list of unique poems in that cluster (only <10 verses)
cluster_to_poems = {}
for cid, group in df_cross.groupby('cluster_id'):
    poems = set([k for k in group['poem_key'] if poem_lengths[k] < 10])
    if poems:
        cluster_to_poems[cid] = poems

# Iterate over clusters
for idx, cluster_row in diverse_clusters.iterrows():
    cid = cluster_row['cluster_id']
    if cid not in cluster_to_poems:
        continue

    poems_in_cluster = list(cluster_to_poems[cid])
    
    shared_cluster_results = []

    # Check all cross-dataset poem pairs for shared clusters
    for p1, p2 in combinations(poems_in_cluster, 2):
        if p1[0] == p2[0]:  # skip same dataset
            continue

        shared = (poem_to_clusters[p1] & poem_to_clusters[p2]) - {cid}
        if shared:
            shared_cluster_results.append((p1, p2, shared))

    # Skip cluster if no cross-dataset pairs share other clusters
    if not shared_cluster_results:
        continue

    # Retrieve verses for this cluster
    cluster_verses = df_cross[df_cross['cluster_id'] == cid]

    print(f"\n=== Cluster {cid} ({len(cluster_verses)} verses, {cluster_row['n_datasets']} datasets) ===")

    for _, verse_row in cluster_verses.iterrows():
        dataset = verse_row['source_dataset']
        verse_text = verse_row['verse']
        poem_id = verse_row['idoriginal_poem']
        print(f"[{dataset}] : {poem_id}: {verse_text}")

    print("\n--- Other clusters shared by cross-dataset poem pairs in this cluster ---")
    for p1, p2, shared_clusters in shared_cluster_results:
        print(f"Poem pair {p1} & {p2} co-occur in clusters: {list(shared_clusters)}")


/tmp/ipykernel_3020693/1581249325.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cross = pd.read_csv(f"{RESULTS_DIR}/concatenated_cross_dataset_clusters.csv")



=== Cluster 14120 (9 verses, 4 datasets) ===
[rhoby] : band3_griechenland_GR72: τοδ ετει τριτω κατωρυξα σωμα
[dbbe] : 24938: τον του τελους τριπηχυν ανθειλου ταφον
[dbbe] : 25552: αυταρ ογε τριτατω ενι ηματι τυμβον ανοιξας
[phi] : 102903: τρισσους ενθαδε θαψε λβω
[phi] : 102903: τρισσους ενθαδε θαψε λβω
[phi] : 876907: τον τριετη τυνβω τωδ υπεθηκε νεκυν
[phi] : 23049: σα ετη ηʹ και μηνας τριων ²⁶τρεις²⁶ ταφεισα
[papyri] : 3581: το τριτον ιμουθης ζμινος εως της ταφης
[papyri] : 23049: σα ετη η και μηνων τριων ταφεισα

--- Other clusters shared by cross-dataset poem pairs in this cluster ---
Poem pair ('phi', '23049') & ('papyri', '23049') co-occur in clusters: [4, 1052]

=== Cluster 12289 (13 verses, 4 datasets) ===
[rhoby] : band2_epigramme_auf_ikonen_Ik31: ορω σε ταφε δειλιω σου την θεαν
[dbbe] : 19610: ορω σε θανατε δειληω σου την θεαν
[dbbe] : 33805: ορω σε θανατε δειλιω σου την θεαν
[phi] : 898761: σειος αμφινοος θιοζοτειος θιο
[phi] : 779322: τον τε θεος παταξαι απορια και
[phi] 

In [2]:
import pandas as pd
from itertools import combinations
from collections import defaultdict

RESULTS_DIR = "full_semantic_results_test"

# ----------------------------
# Load data
# ----------------------------
df_cross = pd.read_csv(f"{RESULTS_DIR}/concatenated_cross_dataset_clusters.csv")
original_df = pd.read_csv("concatenated.csv")  # contains full poems

# Ensure IDs are strings
df_cross['idoriginal_poem'] = df_cross['idoriginal_poem'].astype(str)
original_df['idoriginal_poem'] = original_df['idoriginal_poem'].astype(str)

# Build unique poem key
df_cross['poem_key'] = list(zip(df_cross['source_dataset'], df_cross['idoriginal_poem']))

# ----------------------------
# Compute poem lengths (number of verses)
# ----------------------------
poem_lengths = df_cross.groupby('poem_key').size().to_dict()

# ----------------------------
# Precompute poem -> clusters
# ----------------------------
poem_to_clusters = df_cross.groupby('poem_key')['cluster_id'].apply(set).to_dict()

# ----------------------------
# Precompute cluster -> poems
# ----------------------------
cluster_to_poems = df_cross.groupby('cluster_id')['poem_key'].apply(set).to_dict()

# ----------------------------
# Identify candidate poem pairs that share clusters
# ----------------------------
shared_cluster_counts = defaultdict(int)  # (p1, p2) -> count of shared clusters

# Only consider cross-dataset poem pairs
for cid, poems in cluster_to_poems.items():
    poems = list(poems)
    for p1, p2 in combinations(poems, 2):
        if p1[0] == p2[0]:  # skip same dataset
            continue
        key = tuple(sorted([p1, p2]))
        shared_cluster_counts[key] += 1

# ----------------------------
# Rank poem pairs by shared clusters / total length
# ----------------------------
# Using a fast scoring function: shared clusters divided by total verses
top_pairs = sorted(
    shared_cluster_counts.items(),
    key=lambda x: x[1] / (poem_lengths[x[0][0]] + poem_lengths[x[0][1]]),
    reverse=True
)[:20]  # top 20 pairs

# ----------------------------
# Print full poems for these pairs
# ----------------------------
for (p1, p2), score in top_pairs:
    shared_count = shared_cluster_counts[(p1, p2)]
    print("\n" + "="*80)
    print(f"Poem pair {p1} & {p2} share {shared_count} clusters "
          f"(score={score:.3f}, combined length={poem_lengths[p1]+poem_lengths[p2]})")
    print("="*80)

    for poem_key in [p1, p2]:
        dataset, poem_id = poem_key
        poem_verses = original_df[(original_df['source_dataset'] == dataset) & 
                                  (original_df['idoriginal_poem'] == poem_id)].sort_values('order')
        print(f"\n[{dataset}] Poem ID: {poem_id} ({len(poem_verses)} verses)")
        for _, row in poem_verses.iterrows():
            verse_text = row.get('original_verse', row['verse'])
            print(verse_text)


/tmp/ipykernel_3020693/1656130796.py:10: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cross = pd.read_csv(f"{RESULTS_DIR}/concatenated_cross_dataset_clusters.csv")
/tmp/ipykernel_3020693/1656130796.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  original_df = pd.read_csv("concatenated.csv")  # contains full poems



Poem pair ('dbbe', '32985') & ('phi', '775777') share 1 clusters (score=1.000, combined length=2)

[dbbe] Poem ID: 32985 (1 verses)
εἰς τὸν λουκ(ᾶν) ἄνωθ(εν) τρεῖς στίχ(οι) πάλιν·

[phi] Poem ID: 775777 (2 verses)
.— — ἐκ Τρικορύ[θου]
.[Ἀσκλ]ηπιάδου.

Poem pair ('dbbe', '17824') & ('phi', '775777') share 1 clusters (score=1.000, combined length=2)

[dbbe] Poem ID: 17824 (1 verses)
+ τέλος δευτέρας τῶν τριῶν τῆς τριβίβλου

[phi] Poem ID: 775777 (2 verses)
.— — ἐκ Τρικορύ[θου]
.[Ἀσκλ]ηπιάδου.

Poem pair ('dbbe', '23115') & ('phi', '775777') share 1 clusters (score=1.000, combined length=2)

[dbbe] Poem ID: 23115 (1 verses)
+ τριὰς μονὰς δίδου πονέοντι τεὴν πολυολβον ἀρωγήν

[phi] Poem ID: 775777 (2 verses)
.— — ἐκ Τρικορύ[θου]
.[Ἀσκλ]ηπιάδου.

Poem pair ('papyri', '42748') & ('phi', '775777') share 1 clusters (score=1.000, combined length=2)

[papyri] Poem ID: 42748 (5 verses)
Σύρων
Πτολεμαῖος Ἰου-
λιανοῦ σάκκοι
τρεῖς, γί(νονται) γ´. δ´ (ἔτους)
Ἁθὺρ κϛ´.

[phi] Poem ID: 775777 (2 verses

In [ ]:
import pandas as pd

RESULTS_DIR = "full_semantic_results_test"
df_cross = pd.read_csv(f"{RESULTS_DIR}/cross_dataset_verses_only.csv")

# Cluster IDs you want
cluster_ids = [2452, 2602, 2311]

# Filter rows for these clusters
df_filtered = df_cross[df_cross['cluster_id'].isin(cluster_ids)]

# Group by cluster and print
for cid, cluster_data in df_filtered.groupby('cluster_id'):
    print(f"\n=== Cluster {cid} ({len(cluster_data)} verses) ===\n")
    
    for _, row in cluster_data.iterrows():
        print(f"[{row['source_dataset']}] : {row['verse']}")
    
    print("\n" + "."*50)  

## 2.2 Poem level clustering

In [12]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
from numba import njit
from pathlib import Path
import gc

RESULTS_DIR = Path("full_semantic_results_test")
POEM_RESULTS_DIR = Path("full_semantic_results_cross_dataset_poems")
SCRATCH_DIR = Path("/scratch/gent/vo/000/gvo00042/vsc48660/poem_clustering_temp")
POEM_RESULTS_DIR.mkdir(exist_ok=True)
SCRATCH_DIR.mkdir(exist_ok=True, parents=True)

INPUT_CSV = RESULTS_DIR / "concatenated_cross_dataset_clusters.csv"

SAMPLE_SIZE = 15000
RANDOM_SEED = 42

JACCARD_THRESHOLD_RANGE = (0.01, 0.1, 13)

MAX_WORKERS = 8
BATCH_SIZE = 50000

print("="*80)
print("POEM-LEVEL CLUSTERING BASED ON VERSE CLUSTER JACCARD SIMILARITY")
print("="*80)
print(f"Input: {INPUT_CSV}")
print(f"Sample size: {SAMPLE_SIZE:,}")
print(f"Jaccard threshold range: {JACCARD_THRESHOLD_RANGE}")
print(f"Workers: {MAX_WORKERS}")
print(f"Scratch dir: {SCRATCH_DIR}")
print("="*80)

@njit
def compute_jaccard_similarity(a_clusters, b_clusters):
    a_set = set(a_clusters)
    b_set = set(b_clusters)
    
    intersection = len(a_set & b_set)
    union = len(a_set | b_set)
    
    if union == 0:
        return 0.0
    
    return intersection / union

class FastUnionFind:
    __slots__ = ['parent', 'rank']
    
    def __init__(self, elements):
        self.parent = {e: e for e in elements}
        self.rank = {e: 0 for e in elements}
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        px, py = self.find(x), self.find(y)
        if px == py:
            return False
        if self.rank[px] < self.rank[py]:
            px, py = py, px
        self.parent[py] = px
        if self.rank[px] == self.rank[py]:
            self.rank[px] += 1
        return True
    
    def get_clusters(self):
        clusters = defaultdict(set)
        for elem in self.parent.keys():
            clusters[self.find(elem)].add(elem)
        return dict(clusters)

def reconstruct_poems_from_verses(df):
    print("\nReconstructing poems from verses...")
    
    required_cols = ['idoriginal_poem', 'cluster_id', 'source_dataset']
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"ERROR: '{col}' column not found in input CSV.")
    
    valid_mask = df['cluster_id'] != -1
    df_valid = df[valid_mask].copy()
    
    print(f"  ✓ Total verses: {len(df):,}")
    print(f"  ✓ Valid verses (cluster_id != -1): {len(df_valid):,}")
    
    df_valid['poem_composite_id'] = df_valid['idoriginal_poem'].astype(str) + '___' + df_valid['source_dataset'].astype(str)
    
    if 'order' in df_valid.columns:
        df_valid = df_valid.sort_values(['poem_composite_id', 'order'])
    else:
        df_valid = df_valid.sort_values('poem_composite_id')

    poem_to_clusters = {}
    poem_to_dataset = {}
    poem_to_size = {}
    
    for composite_id, group in df_valid.groupby('poem_composite_id'):
        cluster_sequence = group['cluster_id'].values
        poem_to_clusters[composite_id] = np.unique(cluster_sequence).astype(np.int32)
        poem_to_dataset[composite_id] = group['source_dataset'].iloc[0]
        poem_to_size[composite_id] = len(poem_to_clusters[composite_id])
    
    print(f"  ✓ Reconstructed {len(poem_to_clusters):,} poems")
    print(f"  ✓ Average unique clusters per poem: {np.mean(list(poem_to_size.values())):.1f}")
    
    dataset_counts = defaultdict(int)
    for dataset in poem_to_dataset.values():
        dataset_counts[dataset] += 1
    
    print(f"  ✓ Poems by dataset:")
    for dataset, count in sorted(dataset_counts.items()):
        print(f"      {dataset}: {count:,}")
    
    poems_by_dataset = defaultdict(list)
    for poem_id, dataset in poem_to_dataset.items():
        poems_by_dataset[dataset].append(poem_id)
    
    poem_metadata = {
        'poem_to_size': poem_to_size,
        'poems_by_dataset': dict(poems_by_dataset)
    }
    
    return poem_to_clusters, poem_to_dataset, poem_metadata

def build_cluster_to_poems_index(poem_to_clusters):
    print("\nBuilding cluster-to-poems inverted index...")
    cluster_to_poems = defaultdict(list)
    for poem_id, cluster_array in tqdm(poem_to_clusters.items(), desc="Indexing"):
        for cluster_id in cluster_array:
            cluster_to_poems[int(cluster_id)].append(poem_id)
    
    for cluster_id in cluster_to_poems:
        cluster_to_poems[cluster_id] = np.array(cluster_to_poems[cluster_id])
    
    return dict(cluster_to_poems)

def stratified_sample_poems(poem_to_clusters, poem_to_dataset, poem_to_size, n_sample=10000):
    print(f"\nStratified sampling of {n_sample:,} poems...")
    
    np.random.seed(RANDOM_SEED)
    
    poem_metadata = []
    for poem_id in poem_to_clusters.keys():
        metadata = {
            'poem_id': poem_id,
            'n_verses': poem_to_size[poem_id],
            'source': poem_to_dataset[poem_id]
        }
        poem_metadata.append(metadata)
    
    poem_df = pd.DataFrame(poem_metadata)
    
    poem_df['size_bin'] = pd.cut(poem_df['n_verses'], 
                                  bins=[0, 5, 10, 20, 50, np.inf],
                                  labels=['tiny', 'small', 'medium', 'large', 'huge'])
    
    sample_indices = []
    
    for (source, size_bin), group in poem_df.groupby(['source', 'size_bin']):
        n_in_group = len(group)
        proportion = n_in_group / len(poem_df)
        n_sample_group = max(1, int(n_sample * proportion))
        n_sample_group = min(n_sample_group, n_in_group)
        
        sampled = group.sample(n=n_sample_group, random_state=RANDOM_SEED)
        sample_indices.extend(sampled['poem_id'].tolist())
    
    if len(sample_indices) < n_sample:
        remaining = n_sample - len(sample_indices)
        available = set(poem_to_clusters.keys()) - set(sample_indices)
        if available:
            additional = np.random.choice(list(available), 
                                         size=min(remaining, len(available)), 
                                         replace=False)
            sample_indices.extend(additional)
    
    sample_indices = sample_indices[:n_sample]
    print(f"  ✓ Sampled {len(sample_indices):,} poems")
    
    return sample_indices

def find_cross_dataset_candidate_pairs_batched(poem_to_clusters, poem_to_dataset, cluster_to_poems, poems_by_dataset):
    print("\nFinding cross-dataset candidate pairs (batched)...")
    
    datasets = list(poems_by_dataset.keys())
    pair_file = SCRATCH_DIR / "candidate_pairs.npz"
    
    if pair_file.exists():
        print("  ✓ Loading cached candidate pairs...")
        data = np.load(pair_file)
        return set(zip(data['p1'], data['p2']))
    
    all_pairs = set()
    
    for i, dataset1 in enumerate(datasets):
        for dataset2 in datasets[i+1:]:
            print(f"  Processing {dataset1} × {dataset2}...")
            poems1 = poems_by_dataset[dataset1]
            poems2 = poems_by_dataset[dataset2]
            poems2_set = set(poems2)
            
            batch_pairs = set()
            
            for poem_id in tqdm(poems1, desc=f"  {dataset1}"):
                clusters = poem_to_clusters[poem_id]
                
                candidates = set()
                for cluster_id in clusters:
                    if int(cluster_id) in cluster_to_poems:
                        candidates.update(cluster_to_poems[int(cluster_id)])
                
                candidates = candidates & poems2_set
                
                for other_poem in candidates:
                    pair = tuple(sorted([poem_id, other_poem]))
                    batch_pairs.add(pair)
            
            all_pairs.update(batch_pairs)
            print(f"    Found {len(batch_pairs):,} pairs")
    
    print(f"  ✓ Total candidate pairs: {len(all_pairs):,}")
    
    if all_pairs:
        p1_list, p2_list = zip(*all_pairs)
        np.savez_compressed(pair_file, p1=p1_list, p2=p2_list)
        print(f"  ✓ Cached pairs to {pair_file}")
    
    return all_pairs

def compute_cluster_cohesion(poem_to_clusters, cluster_assignments, max_sample=500):
    poem_ids = list(poem_to_clusters.keys())
    cohesions = []
    
    clusters = defaultdict(list)
    for poem_id in poem_ids:
        cluster_id = cluster_assignments.get(poem_id)
        if cluster_id is not None:
            clusters[cluster_id].append(poem_id)
    
    for cluster_id, cluster_poems in clusters.items():
        if len(cluster_poems) < 2:
            continue
        
        if len(cluster_poems) > 30:
            sampled = np.random.choice(cluster_poems, 30, replace=False)
        else:
            sampled = cluster_poems
        
        overlaps = []
        for i in range(len(sampled)):
            for j in range(i+1, min(i+10, len(sampled))):
                jaccard = compute_jaccard_similarity(
                    poem_to_clusters[sampled[i]], 
                    poem_to_clusters[sampled[j]]
                )
                overlaps.append(jaccard)
        
        if overlaps:
            cohesions.append(np.mean(overlaps))
        
        if len(cohesions) >= max_sample:
            break
    
    return np.mean(cohesions) if cohesions else 0.0

def evaluate_single_config(args):
    jaccard_thresh, sample_poems, poem_to_clusters, candidate_pairs, poem_to_dataset = args
    
    try:
        sample_set = set(sample_poems)
        cross_dataset_pairs = set()
        
        for p1, p2 in candidate_pairs:
            if p1 in sample_set and p2 in sample_set:
                cross_dataset_pairs.add((p1, p2))
        
        uf = FastUnionFind(sample_poems)
        merges = 0
        
        for p1, p2 in cross_dataset_pairs:
            jaccard = compute_jaccard_similarity(
                poem_to_clusters[p1], 
                poem_to_clusters[p2]
            )
            
            if jaccard >= jaccard_thresh:
                if uf.union(p1, p2):
                    merges += 1
        
        poem_clusters = uf.get_clusters()
        cluster_assignments = {}
        for cluster_id, poems in poem_clusters.items():
            for poem in poems:
                cluster_assignments[poem] = cluster_id
        
        n_clusters = len(poem_clusters)
        cluster_sizes = [len(poems) for poems in poem_clusters.values()]
        n_singletons = sum(1 for size in cluster_sizes if size == 1)
        avg_size = np.mean(cluster_sizes)
        max_size = max(cluster_sizes) if cluster_sizes else 0
        
        n_cross_dataset_clusters = 0
        for cluster_id, poems in poem_clusters.items():
            datasets = set(poem_to_dataset.get(p) for p in poems)
            if len(datasets) > 1:
                n_cross_dataset_clusters += 1
        
        cohesion = compute_cluster_cohesion(poem_to_clusters, cluster_assignments)
        
        return {
            'jaccard_threshold': jaccard_thresh,
            'n_clusters': n_clusters,
            'n_singletons': n_singletons,
            'n_cross_dataset_clusters': n_cross_dataset_clusters,
            'avg_cluster_size': avg_size,
            'max_cluster_size': max_size,
            'cohesion': cohesion,
            'merges': merges,
            'n_cross_dataset_pairs': len(cross_dataset_pairs)
        }
    
    except Exception as e:
        print(f"Error at jaccard={jaccard_thresh:.2f}: {e}")
        return None

def grid_search_parameters(poem_to_clusters, poem_to_dataset, poem_to_size, poems_by_dataset):
    print("\n" + "="*80)
    print("GRID SEARCH: JACCARD THRESHOLD")
    print("="*80)

    sample_poems = stratified_sample_poems(poem_to_clusters, poem_to_dataset, 
                                          poem_to_size, SAMPLE_SIZE)
    
    sample_poems_set = set(sample_poems)
    sample_poems_by_dataset = defaultdict(list)
    for poem_id in sample_poems:
        sample_poems_by_dataset[poem_to_dataset[poem_id]].append(poem_id)
    
    cluster_to_poems = build_cluster_to_poems_index({p: poem_to_clusters[p] for p in sample_poems})
    
    candidate_pairs = find_cross_dataset_candidate_pairs_batched(
        {p: poem_to_clusters[p] for p in sample_poems},
        poem_to_dataset,
        cluster_to_poems,
        sample_poems_by_dataset
    )

    jaccard_thresholds = np.linspace(JACCARD_THRESHOLD_RANGE[0], 
                                     JACCARD_THRESHOLD_RANGE[1], 
                                     int(JACCARD_THRESHOLD_RANGE[2]))
    
    print(f"\nParameter grid:")
    print(f"  Jaccard thresholds: {len(jaccard_thresholds)} values from {jaccard_thresholds[0]:.2f} to {jaccard_thresholds[-1]:.2f}")

    args_list = []
    for jaccard_thresh in jaccard_thresholds:
        args_list.append((
            jaccard_thresh, sample_poems, poem_to_clusters,
            candidate_pairs, poem_to_dataset
        ))

    print(f"\nRunning grid search with {MAX_WORKERS} workers...")
    start_time = time.time()
    results = []

    with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(evaluate_single_config, args): args for args in args_list}
        with tqdm(total=len(futures), desc="Grid search") as pbar:
            for future in as_completed(futures):
                result = future.result()
                if result is not None:
                    results.append(result)
                pbar.update(1)

    print(f"✓ Grid search complete in {time.time() - start_time:.1f}s")

    results_df = pd.DataFrame(results)

    def normalize(series):
        min_val = series.min()
        max_val = series.max()
        if max_val - min_val < 1e-10:
            return pd.Series(0.5, index=series.index)
        return (series - min_val) / (max_val - min_val)

    cohesion_score = normalize(results_df['cohesion'])
    
    cross_dataset_ratio = results_df['n_cross_dataset_clusters'] / (results_df['n_clusters'] + 1e-10)
    cross_dataset_score = normalize(cross_dataset_ratio)
    
    singleton_ratio = results_df['n_singletons'] / len(sample_poems)
    balance_score = np.clip(1 - singleton_ratio, 0, 1)

    results_df['quality_score'] = (
        cohesion_score * 0.50 +
        cross_dataset_score * 0.30 +
        balance_score * 0.20
    )

    results_df = results_df.sort_values('quality_score', ascending=False)
    results_csv = POEM_RESULTS_DIR / 'poem_parameter_grid_search.csv'
    results_df.to_csv(results_csv, index=False)
    print(f"✓ Results saved: {results_csv}")

    create_grid_visualizations(results_df)

    best = results_df.iloc[0]
    best_jaccard = float(best['jaccard_threshold'])
    
    print("\n" + "="*80)
    print("🎯 SELECTED CONFIGURATION (HIGHEST QUALITY)")
    print("="*80)
    print(f"Jaccard threshold: {best_jaccard:.3f}")
    print(f"Quality score: {best['quality_score']:.3f}")
    print(f"Cohesion: {best['cohesion']:.3f}")
    print(f"Cross-dataset clusters: {best['n_cross_dataset_clusters']}")

    return best_jaccard, results_df

def create_grid_visualizations(results_df):
    print("\nCreating visualizations...")
    
    sns.set_palette("colorblind")
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle('Poem-Level Cross-Dataset Clustering: Grid Search', fontsize=16, fontweight='bold')
    
    best = results_df.iloc[0]
    
    ax = axes[0, 0]
    ax.plot(results_df['jaccard_threshold'], results_df['quality_score'], 'o-', linewidth=2, markersize=6)
    ax.axvline(best['jaccard_threshold'], color='red', linestyle='--', linewidth=2, label='Best')
    ax.set_xlabel('Jaccard Threshold', fontweight='bold')
    ax.set_ylabel('Quality Score', fontweight='bold')
    ax.set_title('Quality vs Jaccard', fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)
    
    ax = axes[0, 1]
    ax.plot(results_df['jaccard_threshold'], results_df['cohesion'], 'o-', linewidth=2, markersize=6, color='orange')
    ax.axvline(best['jaccard_threshold'], color='red', linestyle='--', linewidth=2)
    ax.set_xlabel('Jaccard Threshold', fontweight='bold')
    ax.set_ylabel('Cohesion', fontweight='bold')
    ax.set_title('Cohesion vs Jaccard', fontweight='bold')
    ax.grid(alpha=0.3)
    
    ax = axes[0, 2]
    ax.plot(results_df['jaccard_threshold'], results_df['n_cross_dataset_clusters'], 'o-', linewidth=2, markersize=6, color='green')
    ax.axvline(best['jaccard_threshold'], color='red', linestyle='--', linewidth=2)
    ax.set_xlabel('Jaccard Threshold', fontweight='bold')
    ax.set_ylabel('Cross-Dataset Clusters', fontweight='bold')
    ax.set_title('Cross-Dataset Clusters vs Jaccard', fontweight='bold')
    ax.grid(alpha=0.3)
    
    ax = axes[1, 0]
    ax.plot(results_df['jaccard_threshold'], results_df['n_clusters'], 'o-', linewidth=2, markersize=6, color='purple')
    ax.axvline(best['jaccard_threshold'], color='red', linestyle='--', linewidth=2)
    ax.set_xlabel('Jaccard Threshold', fontweight='bold')
    ax.set_ylabel('Total Clusters', fontweight='bold')
    ax.set_title('Total Clusters vs Jaccard', fontweight='bold')
    ax.grid(alpha=0.3)
    
    ax = axes[1, 1]
    ax.scatter(results_df['cohesion'], results_df['n_cross_dataset_clusters'],
              c=results_df['quality_score'], cmap='RdYlGn', s=100, edgecolors='black')
    ax.scatter(best['cohesion'], best['n_cross_dataset_clusters'],
              color='red', s=300, marker='*', edgecolors='black', linewidth=2, zorder=10)
    ax.set_xlabel('Cohesion', fontweight='bold')
    ax.set_ylabel('Cross-Dataset Clusters', fontweight='bold')
    ax.set_title('Cohesion vs Cross-Dataset', fontweight='bold')
    ax.grid(alpha=0.3)
    
    ax = axes[1, 2]
    ax.hist(results_df['quality_score'], bins=15, color='#0173B2', alpha=0.7, edgecolor='black')
    ax.axvline(best['quality_score'], color='red', linestyle='--', linewidth=2, label=f"Best: {best['quality_score']:.3f}")
    ax.set_xlabel('Quality Score', fontweight='bold')
    ax.set_ylabel('Frequency', fontweight='bold')
    ax.set_title('Score Distribution', fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)
    
    plt.tight_layout()
    plot_path = POEM_RESULTS_DIR / 'poem_grid_search_comprehensive.png'
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"✓ Visualization saved: {plot_path}")
    plt.close()

def cluster_all_poems_batched(poem_to_clusters, poem_to_dataset, poems_by_dataset, jaccard_threshold):
    print("\n" + "="*80)
    print("CLUSTERING ALL POEMS (BATCHED, CROSS-DATASET ONLY)")
    print("="*80)
    print(f"Jaccard threshold: {jaccard_threshold:.3f}")
    
    cluster_to_poems = build_cluster_to_poems_index(poem_to_clusters)
    
    candidate_pairs = find_cross_dataset_candidate_pairs_batched(
        poem_to_clusters, poem_to_dataset, cluster_to_poems, poems_by_dataset
    )
    
    print(f"\nClustering {len(poem_to_clusters):,} poems...")
    poem_ids = list(poem_to_clusters.keys())
    uf = FastUnionFind(poem_ids)
    
    merges = 0
    batches = [list(candidate_pairs)[i:i+BATCH_SIZE] for i in range(0, len(candidate_pairs), BATCH_SIZE)]
    
    for batch in tqdm(batches, desc="Processing batches"):
        for p1, p2 in batch:
            jaccard = compute_jaccard_similarity(
                poem_to_clusters[p1], 
                poem_to_clusters[p2]
            )
            
            if jaccard >= jaccard_threshold:
                if uf.union(p1, p2):
                    merges += 1
    
    print(f"  ✓ Performed {merges:,} merges")
    
    poem_clusters = uf.get_clusters()
    cluster_assignments = {}
    for cluster_id, poems in poem_clusters.items():
        for poem in poems:
            cluster_assignments[poem] = cluster_id
    
    n_clusters = len(poem_clusters)
    cluster_sizes = [len(poems) for poems in poem_clusters.values()]
    n_singletons = sum(1 for size in cluster_sizes if size == 1)
    
    n_cross_dataset_clusters = 0
    for cluster_id, poems in poem_clusters.items():
        datasets = set(poem_to_dataset.get(p) for p in poems)
        if len(datasets) > 1:
            n_cross_dataset_clusters += 1
    
    print(f"\n  ✓ Total poem clusters: {n_clusters:,}")
    print(f"  ✓ Cross-dataset clusters: {n_cross_dataset_clusters:,}")
    print(f"  ✓ Singleton poems: {n_singletons:,}")
    print(f"  ✓ Avg cluster size: {np.mean(cluster_sizes):.2f}")
    print(f"  ✓ Max cluster size: {max(cluster_sizes)}")
    
    return cluster_assignments, poem_clusters

def main():
    start_time = time.time()
    
    print(f"\nLoading data from: {INPUT_CSV}")
    df = pd.read_csv(INPUT_CSV)
    
    print(f"  ✓ Loaded {len(df):,} verses")
    
    poem_to_clusters, poem_to_dataset, poem_metadata = reconstruct_poems_from_verses(df)
    poem_to_size = poem_metadata['poem_to_size']
    poems_by_dataset = poem_metadata['poems_by_dataset']
    
    best_jaccard, grid_results = grid_search_parameters(
        poem_to_clusters, poem_to_dataset, poem_to_size, poems_by_dataset
    )
    
    del grid_results
    gc.collect()
    
    cluster_assignments, poem_clusters = cluster_all_poems_batched(
        poem_to_clusters, poem_to_dataset, poems_by_dataset, best_jaccard
    )
    
    df['poem_composite_id'] = df['idoriginal_poem'].astype(str) + '___' + df['source_dataset'].astype(str)
    df['poem_cluster_id'] = df['poem_composite_id'].map(cluster_assignments)
    
    cross_dataset_cluster_ids = set()
    for cluster_id, poems in poem_clusters.items():
        datasets = set(poem_to_dataset.get(p) for p in poems)
        if len(datasets) > 1:
            cross_dataset_cluster_ids.add(cluster_id)
    
    df['is_cross_dataset_poem_cluster'] = df['poem_cluster_id'].isin(cross_dataset_cluster_ids)
    
    output_csv = POEM_RESULTS_DIR / 'poems_clustered_by_verse_jaccard.csv'
    df.to_csv(output_csv, index=False)
    print(f"\n✓ Results saved: {output_csv}")
    
    summary = {
        'n_verses': len(df),
        'n_poems': len(poem_to_clusters),
        'n_datasets': len(set(poem_to_dataset.values())),
        'best_jaccard_threshold': best_jaccard,
        'n_poem_clusters': len(set(cluster_assignments.values())),
        'n_cross_dataset_clusters': len(cross_dataset_cluster_ids),
        'n_poems_in_cross_dataset_clusters': sum(df['is_cross_dataset_poem_cluster']),
        'processing_time_minutes': (time.time() - start_time) / 60
    }
    
    pd.DataFrame([summary]).to_csv(POEM_RESULTS_DIR / 'poem_clustering_summary.csv', index=False)
    
    print("\n" + "="*80)
    print("✅ POEM-LEVEL CLUSTERING COMPLETE")
    print("="*80)
    print(f"Time: {summary['processing_time_minutes']:.1f} minutes")
    print(f"Cross-dataset poem clusters: {summary['n_cross_dataset_clusters']:,}")
    print(f"Poems in cross-dataset clusters: {summary['n_poems_in_cross_dataset_clusters']:,}")
    print(f"All results saved to: {POEM_RESULTS_DIR}/")
    print("="*80)

if __name__ == "__main__":
    main()

POEM-LEVEL CLUSTERING BASED ON VERSE CLUSTER JACCARD SIMILARITY
Input: full_semantic_results_test/concatenated_cross_dataset_clusters.csv
Sample size: 15,000
Jaccard threshold range: (0.01, 0.1, 13)
Workers: 8
Scratch dir: /scratch/gent/vo/000/gvo00042/vsc48660/poem_clustering_temp

Loading data from: full_semantic_results_test/concatenated_cross_dataset_clusters.csv


/tmp/ipykernel_3761598/1156764358.py:556: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(INPUT_CSV)


  ✓ Loaded 1,536,616 verses

Reconstructing poems from verses...
  ✓ Total verses: 1,536,616
  ✓ Valid verses (cluster_id != -1): 1,536,616
  ✓ Reconstructed 174,646 poems
  ✓ Average unique clusters per poem: 8.1
  ✓ Poems by dataset:
      dbbe: 12,615
      papyri: 51,083
      phi: 110,156
      rhoby: 792

GRID SEARCH: JACCARD THRESHOLD

Stratified sampling of 15,000 poems...
  ✓ Sampled 15,000 poems

Building cluster-to-poems inverted index...


Indexing: 100%|██████████| 15000/15000 [00:00<00:00, 265990.89it/s]


Finding cross-dataset candidate pairs (batched)...
  ✓ Loading cached candidate pairs...


...

✓ Grid search complete in 379.9s
✓ Results saved: full_semantic_results_cross_dataset_poems/poem_parameter_grid_search.csv

Creating visualizations...
✓ Visualization saved: full_semantic_results_cross_dataset_poems/poem_grid_search_comprehensive.png

🎯 SELECTED CONFIGURATION (HIGHEST QUALITY)
Jaccard threshold: 0.018
Quality score: 0.516
Cohesion: 0.320
Cross-dataset clusters: 10.0

CLUSTERING ALL POEMS (BATCHED, CROSS-DATASET ONLY)
Jaccard threshold: 0.018

Building cluster-to-poems inverted index...


Indexing: 100%|██████████| 174646/174646 [00:01<00:00, 134668.16it/s]



Finding cross-dataset candidate pairs (batched)...
  ✓ Loading cached candidate pairs...

Clustering 174,646 poems...


Processing batches: 100%|██████████| 47/47 [00:07<00:00,  6.32it/s]


  ✓ Performed 12,447 merges

  ✓ Total poem clusters: 162,199
  ✓ Cross-dataset clusters: 18
  ✓ Singleton poems: 162,181
  ✓ Avg cluster size: 1.08
  ✓ Max cluster size: 12428

✓ Results saved: full_semantic_results_cross_dataset_poems/poems_clustered_by_verse_jaccard.csv

✅ POEM-LEVEL CLUSTERING COMPLETE
Time: 7.2 minutes
Cross-dataset poem clusters: 18
Poems in cross-dataset clusters: 128,112
All results saved to: full_semantic_results_cross_dataset_poems/


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the CSV
df = pd.read_csv('full_semantic_results/poem_parameter_grid_search.csv')

# If there's only one unique min_shared value, we can plot quality vs threshold
unique_min_shared = df['min_shared'].unique()
if len(unique_min_shared) > 1:
    print("Multiple min_shared values detected, consider using a heatmap instead.")
else:
    # Sort by threshold for plotting
    df = df.sort_values('threshold')
    
    plt.figure(figsize=(10, 6))
    sns.lineplot(
        data=df,
        x='threshold',
        y='quality_score',
        marker='o',
        linewidth=2,
        markersize=8
    )
    
    plt.title(f'Quality Score vs Threshold (min_shared={unique_min_shared[0]})',
              fontsize=16, fontweight='bold')
    plt.xlabel('Cosine Similarity Threshold', fontsize=14, fontweight='bold')
    plt.ylabel('Quality Score', fontsize=14, fontweight='bold')
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(alpha=0.3)
    
    plt.tight_layout()
    # Optionally save the figure
    plt.savefig('full_semantic_results/poem_quality_vs_threshold.png', dpi=300, bbox_inches='tight')
    plt.show()


In [14]:
import pandas as pd
df=pd.read_csv('full_semantic_results_cross_dataset_poems/poems_clustered_by_verse_jaccard.csv')
import pandas as pd

def get_filtered_clusters_cross_source_large(df, max_poem_verses=100, min_poems=2, max_poems=200):
    """
    Efficient filtering for large-scale data:
      - poems with <= max_poem_verses
      - clusters with size between min_poems and max_poems
      - only clusters where each poem comes from a different source_dataset
    Returns filtered dataframe and dict: cluster_id -> list of poem_ids
    """
    # Step 1: Count verses per poem
    poem_verse_counts = df.groupby('idoriginal_poem')['verse'].size()
    valid_poems = poem_verse_counts[poem_verse_counts <= max_poem_verses].index

    # Step 2: Filter dataframe to valid poems
    df_valid = df[df['idoriginal_poem'].isin(valid_poems)]

    # Step 3: Compute cluster-level info efficiently
    cluster_poem_df = df_valid.groupby('poem_cluster_id').agg(
        poems=('idoriginal_poem', 'nunique'),
        unique_sources=('source_dataset', 'nunique')
    ).reset_index()

    # Step 4: Filter clusters by size and unique sources
    valid_clusters = cluster_poem_df[
        (cluster_poem_df['poems'] >= min_poems) &
        (cluster_poem_df['poems'] <= max_poems) &
        (cluster_poem_df['poems'] == cluster_poem_df['unique_sources'])
    ]['poem_cluster_id'].values

    # Step 5: Filter dataframe for these clusters
    df_final = df_valid[df_valid['poem_cluster_id'].isin(valid_clusters)]

    # Step 6: Build cluster -> poems mapping
    clusters_dict = df_final.groupby('poem_cluster_id')['idoriginal_poem'].unique().to_dict()

    return df_final, clusters_dict


# ===================== Usage =====================

filtered_df, clusters_dict = get_filtered_clusters_cross_source_large(df)

# Print first 5 clusters with their source_dataset
for i, (cluster_id, poem_ids) in enumerate(clusters_dict.items()):
    if i >= 50:
        break
    print(f"\nCluster {cluster_id} | Poems: {len(poem_ids)}")
    poem_subset = filtered_df[filtered_df['poem_cluster_id'] == cluster_id][['idoriginal_poem','verse','source_dataset']]
    for pid in poem_ids:
        verses = poem_subset[poem_subset['idoriginal_poem'] == pid]
        print(f"  Poem {pid} ({len(verses)} verses):")
        for _, row in verses.iterrows():
            print(f"    {row['verse']}  [source: {row['source_dataset']}]")
    print("-"*60)



/tmp/ipykernel_3761598/2819930227.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('full_semantic_results_cross_dataset_poems/poems_clustered_by_verse_jaccard.csv')



Cluster 113143___phi | Poems: 2
  Poem 113143 (1 verses):
    καρομεμφιτης ²⁷ελληνομεμφιτης²⁷  [source: phi]
  Poem 116832 (1 verses):
    ονικηφοραα μηνιμεσορηκβινδικτιονοςη  [source: papyri]
------------------------------------------------------------

Cluster 116771___papyri | Poems: 2
  Poem 900065 (1 verses):
    θευδωρη ευφιλητου  [source: phi]
  Poem 116771 (1 verses):
    λεμαιωυπερτρυγ  [source: papyri]
------------------------------------------------------------

Cluster 193635___phi | Poems: 2
  Poem 193635 (2 verses):
    tranquilla per annos  [source: phi]
    aemulaque in cunctis  [source: phi]
  Poem 73252 (3 verses):
    olius ca  [source: papyri]
    abban agens midpunctus d ca  [source: papyri]
    donatus cu ca  [source: papyri]
------------------------------------------------------------

Cluster 23372___dbbe | Poems: 2
  Poem 23372 (1 verses):
    ητα δ αιας πολεμιζε μονωι μονος εκτορι διωι  [source: dbbe]
  Poem 36235 (7 verses):
    ς ενομιζον γαρ οτι ουκ ανηλθε

In [13]:
import pandas as pd

def get_filtered_clusters_cross_source_large(df, min_poem_verses=2, max_poem_verses=100, min_poems=2, max_poems=200):
    """
    Efficient filtering for large-scale data:
      - poems with >= min_poem_verses and <= max_poem_verses
      - clusters with size between min_poems and max_poems
      - only clusters where each poem comes from a different source_dataset
    Returns filtered dataframe and dict: cluster_id -> list of poem_ids
    """
    # Step 1: Count verses per poem
    poem_verse_counts = df.groupby('idoriginal_poem')['verse'].size()
    valid_poems = poem_verse_counts[
        (poem_verse_counts >= min_poem_verses) & 
        (poem_verse_counts <= max_poem_verses)
    ].index

    # Step 2: Filter dataframe to valid poems
    df_valid = df[df['idoriginal_poem'].isin(valid_poems)]

    # Step 3: Compute cluster-level info efficiently
    cluster_poem_df = df_valid.groupby('poem_cluster_id').agg(
        poems=('idoriginal_poem', 'nunique'),
        unique_sources=('source_dataset', 'nunique')
    ).reset_index()

    # Step 4: Filter clusters by size and unique sources
    valid_clusters = cluster_poem_df[
        (cluster_poem_df['poems'] >= min_poems) &
        (cluster_poem_df['poems'] <= max_poems) &
        (cluster_poem_df['poems'] == cluster_poem_df['unique_sources'])
    ]['poem_cluster_id'].values

    # Step 5: Filter dataframe for these clusters
    df_final = df_valid[df_valid['poem_cluster_id'].isin(valid_clusters)]

    # Step 6: Build cluster -> poems mapping
    clusters_dict = df_final.groupby('poem_cluster_id')['idoriginal_poem'].unique().to_dict()

    return df_final, clusters_dict

# ===================== Usage =====================

filtered_df, clusters_dict = get_filtered_clusters_cross_source_large(df, min_poem_verses=2)

# Print first 5 clusters with their source_dataset
for i, (cluster_id, poem_ids) in enumerate(clusters_dict.items()):
    if i >= 5:  # limit to first 5 clusters for demo
        break
    print(f"\nCluster {cluster_id} | Poems: {len(poem_ids)}")
    poem_subset = filtered_df[filtered_df['poem_cluster_id'] == cluster_id][['idoriginal_poem','verse','source_dataset']]
    for pid in poem_ids:
        verses = poem_subset[poem_subset['idoriginal_poem'] == pid]
        print(f"  Poem {pid} ({len(verses)} verses):")
        for _, row in verses.iterrows():
            print(f"    {row['verse']}  [source: {row['source_dataset']}]")
    print("-"*60)



Cluster 116206___papyri | Poems: 2
  Poem 127710 (2 verses):
    ανδρι μουσικω χρηστω  [source: phi]
    φιλιππω ερμιονη μν  [source: phi]
  Poem 116886 (2 verses):
    απο μηνα πρεσβυτερου  [source: papyri]
    ονικηφορααμηνι θωθ  [source: papyri]
------------------------------------------------------------

Cluster 117092___papyri | Poems: 2
  Poem 763310 (4 verses):
    μηνις υπερ του υιου  [source: phi]
    αριστοβουλου ⁵⁶ ευχην  [source: phi]
    μηνις υπερ του υιου  [source: phi]
    αριστοβουλου ⁵⁶ ευχην  [source: phi]
  Poem 117092 (2 verses):
    ονικαιφορεαημισυ  [source: papyri]
    μηνιμεσορη ινδικτιονος  [source: papyri]
------------------------------------------------------------

Cluster 128136___phi | Poems: 2
  Poem band3_türkei_TR6 (2 verses):
    τρομω προβλεπε την θειαν λειτουργιαν  [source: rhoby]
    δυσιν ταοισιν ωραισμενοις  [source: rhoby]
  Poem 128136 (2 verses):
    δογματι βουλης δημου  [source: phi]
    ων δυο γαιον ιουλιον  [source: phi]
----------------

## 2.3 poem level embeddings

In [15]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from collections import defaultdict, Counter
import time
import pickle
import gzip
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import igraph as ig
import leidenalg as la
import logging
from concurrent.futures import ProcessPoolExecutor, as_completed
import faiss
import gc

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger(__name__)

CHECKPOINT_DIR = Path("/scratch/gent/vo/000/gvo00042/vsc48660/poem_embeddings_checkpoints")
RESULTS_DIR = Path("full_semantic_results_poem_level")
CHECKPOINT_DIR.mkdir(exist_ok=True, parents=True)
RESULTS_DIR.mkdir(exist_ok=True)

SAMPLE_SIZE = 10000
N_BOOTSTRAP = 2
STABILITY_PAIRS = 2000
MAX_WORKERS = 16
BATCH_SIZE = 64
N_NEIGHBORS = 100
FAISS_NPROBE = 16

logger.info("="*80)
logger.info("POEM-LEVEL CLUSTERING VIA AGGREGATED EMBEDDINGS")
logger.info("="*80)

def aggregate_verse_embeddings_to_poems(df, verse_embeddings):
    logger.info("\n" + "="*80)
    logger.info("AGGREGATING VERSE EMBEDDINGS TO POEMS")
    logger.info("="*80)
    
    df['poem_composite_id'] = df['idoriginal_poem'].astype(str) + '___' + df['source_dataset'].astype(str)
    
    poem_to_verse_indices = defaultdict(list)
    for idx, poem_id in enumerate(df['poem_composite_id']):
        poem_to_verse_indices[poem_id].append(idx)
    
    logger.info(f"Found {len(poem_to_verse_indices):,} unique poems")
    
    poem_embeddings = []
    poem_ids = []
    poem_datasets = []
    poem_sizes = []
    
    checkpoint_file = CHECKPOINT_DIR / 'poem_embeddings.npz'
    metadata_file = CHECKPOINT_DIR / 'poem_metadata.pkl.gz'
    
    if checkpoint_file.exists() and metadata_file.exists():
        logger.info("Loading poem embeddings from checkpoint...")
        data = np.load(checkpoint_file)
        poem_embeddings = data['embeddings']
        
        with gzip.open(metadata_file, 'rb') as f:
            metadata = pickle.load(f)
        
        poem_ids = metadata['poem_ids']
        poem_datasets = metadata['poem_datasets']
        poem_sizes = metadata['poem_sizes']
        
        logger.info(f"✓ Loaded {len(poem_embeddings):,} poem embeddings")
        
    else:
        logger.info("Computing poem embeddings via mean pooling...")
        
        for poem_id in tqdm(sorted(poem_to_verse_indices.keys()), desc="Aggregating"):
            verse_indices = poem_to_verse_indices[poem_id]
            verse_embs = verse_embeddings[verse_indices]
            
            poem_emb = np.mean(verse_embs, axis=0)
            poem_emb = poem_emb / (np.linalg.norm(poem_emb) + 1e-8)
            
            poem_embeddings.append(poem_emb)
            poem_ids.append(poem_id)
            
            dataset = df.iloc[verse_indices[0]]['source_dataset']
            poem_datasets.append(dataset)
            poem_sizes.append(len(verse_indices))
        
        poem_embeddings = np.array(poem_embeddings, dtype=np.float32)
        
        logger.info(f"✓ Created {len(poem_embeddings):,} poem embeddings")
        logger.info(f"✓ Average verses per poem: {np.mean(poem_sizes):.1f}")
        
        logger.info("Saving checkpoint...")
        np.savez_compressed(checkpoint_file, embeddings=poem_embeddings)
        
        metadata = {
            'poem_ids': poem_ids,
            'poem_datasets': poem_datasets,
            'poem_sizes': poem_sizes
        }
        with gzip.open(metadata_file, 'wb') as f:
            pickle.dump(metadata, f, protocol=pickle.HIGHEST_PROTOCOL)
        
        logger.info("✓ Checkpoint saved")
    
    dataset_counts = Counter(poem_datasets)
    logger.info(f"\nPoems by dataset:")
    for dataset, count in sorted(dataset_counts.items()):
        logger.info(f"  {dataset}: {count:,}")
    
    return poem_embeddings, poem_ids, poem_datasets, poem_sizes

def stratified_sample_poems(poem_datasets, poem_sizes, n_sample=10000):
    logger.info(f"\nStratified sampling of {n_sample:,} poems...")
    
    df_meta = pd.DataFrame({
        'index': range(len(poem_datasets)),
        'dataset': poem_datasets,
        'size': poem_sizes
    })
    
    df_meta['size_bin'] = pd.cut(df_meta['size'], 
                                  bins=[0, 5, 10, 20, 50, np.inf],
                                  labels=['tiny', 'small', 'medium', 'large', 'huge'])
    
    sample_indices = []
    
    for (dataset, size_bin), group in df_meta.groupby(['dataset', 'size_bin']):
        n_in_group = len(group)
        proportion = n_in_group / len(df_meta)
        n_sample_group = max(1, int(n_sample * proportion))
        n_sample_group = min(n_sample_group, n_in_group)
        
        sampled = group.sample(n=n_sample_group, random_state=42)
        sample_indices.extend(sampled['index'].tolist())
    
    if len(sample_indices) < n_sample:
        remaining = n_sample - len(sample_indices)
        available = set(range(len(poem_datasets))) - set(sample_indices)
        if available:
            additional = np.random.choice(list(available), 
                                         size=min(remaining, len(available)), 
                                         replace=False)
            sample_indices.extend(additional)
    
    sample_indices = sorted(sample_indices[:n_sample])
    logger.info(f"✓ Sampled {len(sample_indices):,} poems")
    
    return np.array(sample_indices)

def compute_stability_fast(partitions_list, n_nodes, sample_size=STABILITY_PAIRS):
    if n_nodes < 100:
        return 0.0
    
    n_partitions = len(partitions_list)
    n_sample_pairs = min(sample_size, n_nodes * (n_nodes - 1) // 2)
    
    pairs_i = np.random.randint(0, n_nodes, n_sample_pairs)
    pairs_j = np.random.randint(0, n_nodes, n_sample_pairs)
    valid = pairs_i != pairs_j
    pairs_i = pairs_i[valid][:n_sample_pairs]
    pairs_j = pairs_j[valid][:n_sample_pairs]
    
    coclustering = 0
    for membership in partitions_list:
        membership_arr = np.array(membership)
        matches = membership_arr[pairs_i] == membership_arr[pairs_j]
        coclustering += np.sum(matches)
    
    stability = coclustering / (len(pairs_i) * n_partitions)
    return stability

def evaluate_single_combination(args):
    threshold_pct, threshold, resolution, edges_data, weights_data, dataset_map, n_nodes = args
    
    try:
        edge_mask = weights_data >= threshold
        edges = edges_data[edge_mask]
        weights = weights_data[edge_mask]
        
        if len(edges) == 0:
            return None
        
        g = ig.Graph(n=n_nodes, edges=edges.tolist(), directed=False)
        g.es['weight'] = weights.tolist()
        
        bootstrap_memberships = []
        bootstrap_qualities = []
        
        for seed in range(N_BOOTSTRAP):
            partition = la.find_partition(
                g,
                la.CPMVertexPartition,
                weights='weight',
                resolution_parameter=resolution,
                n_iterations=5,
                seed=seed
            )
            bootstrap_memberships.append(partition.membership)
            bootstrap_qualities.append(partition.quality())
        
        membership = bootstrap_memberships[0]
        n_clusters = len(set(membership))
        
        if n_clusters == 0 or n_clusters == n_nodes:
            return None
        
        stability = compute_stability_fast(bootstrap_memberships, n_nodes)
        avg_size = n_nodes / n_clusters
        
        cluster_datasets = defaultdict(set)
        cluster_sizes = defaultdict(int)
        
        for idx, cid in enumerate(membership):
            cluster_datasets[cid].add(dataset_map[idx])
            cluster_sizes[cid] += 1
        
        n_cross_clusters = sum(1 for datasets in cluster_datasets.values() 
                              if len(datasets) > 1)
        pct_cross_clusters = (n_cross_clusters / n_clusters * 100) if n_clusters > 0 else 0
        n_singleton = sum(1 for size in cluster_sizes.values() if size == 1)
        
        n_multi_dataset = sum(1 for datasets in cluster_datasets.values() if len(datasets) >= 3)
        
        avg_quality = np.mean(bootstrap_qualities)
        modularity = g.modularity(membership, weights='weight')
        
        return {
            'threshold_percentile': threshold_pct,
            'threshold_value': threshold,
            'resolution': resolution,
            'n_edges': g.ecount(),
            'graph_density': 2*g.ecount()/(n_nodes*(n_nodes-1)) if n_nodes > 1 else 0,
            'stability': stability,
            'n_clusters': n_clusters,
            'n_singleton': n_singleton,
            'avg_cluster_size': avg_size,
            'n_cross_clusters': n_cross_clusters,
            'pct_cross_clusters': pct_cross_clusters,
            'n_multi_dataset_clusters': n_multi_dataset,
            'avg_quality': avg_quality,
            'modularity': modularity
        }
    
    except Exception as e:
        logger.warning(f"Error at P{threshold_pct}, res={resolution:.2e}: {e}")
        return None

def grid_search_parameters(sample_embeddings, sample_datasets):
    logger.info("\n" + "="*80)
    logger.info("PARAMETER GRID SEARCH")
    logger.info("="*80)
    
    logger.info("Building FAISS index for sample...")
    dimension = sample_embeddings.shape[1]
    index_sample = faiss.IndexFlatIP(dimension)
    faiss.normalize_L2(sample_embeddings)
    index_sample.add(sample_embeddings)
    
    k = min(200, len(sample_embeddings) - 1)
    similarities, indices = index_sample.search(sample_embeddings, k)
    
    logger.info("Precomputing cross-dataset similarities...")
    cross_similarities = []
    for i in range(len(sample_embeddings)):
        neighbor_datasets = sample_datasets[indices[i, 1:]]
        cross_mask = neighbor_datasets != sample_datasets[i]
        cross_similarities.extend(similarities[i, 1:][cross_mask])
    
    cross_similarities = np.array(cross_similarities)
    logger.info(f"✓ Collected {len(cross_similarities):,} cross-dataset pairs")
    
    all_percentiles = list(range(50, 100))
    threshold_lookup = {p: np.percentile(cross_similarities, p) for p in all_percentiles}
    
    logger.info(f"Threshold range: {threshold_lookup[50]:.4f} (P50) to {threshold_lookup[99]:.4f} (P99)")
    
    logger.info("Precomputing edge structures...")
    all_edges_list = []
    all_weights_list = []
    
    for i in range(len(sample_embeddings)):
        dataset_i = sample_datasets[i]
        neighbors = indices[i, 1:]
        sims = similarities[i, 1:]
        
        valid_mask = neighbors > i
        valid_neighbors = neighbors[valid_mask]
        valid_sims = sims[valid_mask]
        
        if len(valid_neighbors) > 0:
            neighbor_datasets = sample_datasets[valid_neighbors]
            cross_mask = neighbor_datasets != dataset_i
            
            final_neighbors = valid_neighbors[cross_mask]
            final_sims = valid_sims[cross_mask]
            
            for j, sim in zip(final_neighbors, final_sims):
                all_edges_list.append([i, j])
                all_weights_list.append(sim)
    
    all_edges = np.array(all_edges_list, dtype=np.int32)
    all_weights = np.array(all_weights_list, dtype=np.float32)
    
    logger.info(f"✓ Precomputed {len(all_edges):,} edges")
    
    threshold_percentiles_coarse = [90, 93, 96, 98]
    resolutions_coarse = np.logspace(-6, -1, 8)
    
    logger.info(f"\nCoarse grid: {len(threshold_percentiles_coarse)} thresholds × {len(resolutions_coarse)} resolutions")
    
    coarse_args = []
    for threshold_pct in threshold_percentiles_coarse:
        threshold = threshold_lookup.get(threshold_pct, np.percentile(cross_similarities, threshold_pct))
        for resolution in resolutions_coarse:
            coarse_args.append((
                threshold_pct, threshold, resolution,
                all_edges, all_weights, sample_datasets, len(sample_embeddings)
            ))
    
    logger.info(f"Running coarse sweep with {MAX_WORKERS} workers...")
    coarse_results = []
    with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(evaluate_single_combination, args): args for args in coarse_args}
        with tqdm(total=len(futures), desc="Coarse sweep") as pbar:
            for future in as_completed(futures):
                result = future.result()
                if result is not None:
                    coarse_results.append(result)
                pbar.update(1)
    
    logger.info(f"✓ Coarse sweep: {len(coarse_results)} valid results")
    
    if len(coarse_results) == 0:
        logger.error("No valid results!")
        raise ValueError("Grid search failed")
    
    coarse_df = pd.DataFrame(coarse_results)
    best_coarse = coarse_df.loc[coarse_df['stability'].idxmax()]
    
    logger.info(f"\nBest coarse: P{best_coarse['threshold_percentile']}, res={best_coarse['resolution']:.2e}")
    logger.info(f"  Stability: {best_coarse['stability']:.3f}")
    logger.info(f"  Cross-dataset clusters: {best_coarse['n_cross_clusters']}")
    
    thresh_fine = [max(50, int(best_coarse['threshold_percentile']) - 5),
                   max(50, int(best_coarse['threshold_percentile']) - 2),
                   int(best_coarse['threshold_percentile']),
                   min(99, int(best_coarse['threshold_percentile']) + 2),
                   min(99, int(best_coarse['threshold_percentile']) + 5)]
    thresh_fine = sorted(list(set(thresh_fine)))
    
    log_res = np.log10(best_coarse['resolution'])
    res_fine = np.logspace(log_res - 0.5, log_res + 0.5, 7)
    
    logger.info(f"\nFine grid: {len(thresh_fine)} thresholds × {len(res_fine)} resolutions")
    
    fine_args = []
    for threshold_pct in thresh_fine:
        threshold = threshold_lookup.get(threshold_pct, np.percentile(cross_similarities, threshold_pct))
        for resolution in res_fine:
            already_tested = any(
                np.isclose(r['threshold_percentile'], threshold_pct) and 
                np.isclose(r['resolution'], resolution, rtol=0.1)
                for r in coarse_results
            )
            if not already_tested:
                fine_args.append((
                    threshold_pct, threshold, resolution,
                    all_edges, all_weights, sample_datasets, len(sample_embeddings)
                ))
    
    logger.info(f"Running fine sweep...")
    fine_results = []
    with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(evaluate_single_combination, args): args for args in fine_args}
        with tqdm(total=len(futures), desc="Fine sweep") as pbar:
            for future in as_completed(futures):
                result = future.result()
                if result is not None:
                    fine_results.append(result)
                pbar.update(1)
    
    logger.info(f"✓ Fine sweep: {len(fine_results)} valid results")
    
    all_results = coarse_results + fine_results
    sweep_df = pd.DataFrame(all_results)
    sweep_df = sweep_df.sort_values('stability', ascending=False)
    sweep_df.to_csv(RESULTS_DIR / 'poem_parameter_sweep.csv', index=False)
    
    best_params = sweep_df.iloc[0]
    
    logger.info("\n" + "="*80)
    logger.info("TOP 5 CONFIGURATIONS")
    logger.info("="*80)
    for idx, (_, row) in enumerate(sweep_df.head(5).iterrows(), 1):
        logger.info(f"\n#{idx}. Threshold: P{row['threshold_percentile']} ({row['threshold_value']:.4f}), "
                   f"Resolution: {row['resolution']:.2e}")
        logger.info(f"     Stability: {row['stability']:.3f}")
        logger.info(f"     Clusters: {row['n_clusters']}, Cross-dataset: {row['n_cross_clusters']} ({row['pct_cross_clusters']:.1f}%)")
        logger.info(f"     Multi-dataset (3+): {row['n_multi_dataset_clusters']}")
    
    logger.info("\n" + "="*80)
    logger.info("SELECTED PARAMETERS")
    logger.info("="*80)
    logger.info(f"Threshold: P{best_params['threshold_percentile']} = {best_params['threshold_value']:.4f}")
    logger.info(f"Resolution: {best_params['resolution']:.6e}")
    logger.info(f"Stability: {best_params['stability']:.3f}")
    logger.info("="*80)
    
    create_visualization(sweep_df, best_params)
    
    return best_params['threshold_value'], best_params['resolution']

def create_visualization(sweep_df, best_params):
    logger.info("\nCreating visualization...")
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle('Poem-Level Clustering: Parameter Sweep', fontsize=16, fontweight='bold')
    
    pivot_data = sweep_df.pivot_table(
        values='stability',
        index='resolution',
        columns='threshold_percentile',
        aggfunc='first'
    )
    pivot_data.index = [f"{res:.3e}" for res in pivot_data.index]
    
    ax = axes[0, 0]
    sns.heatmap(pivot_data, annot=True, fmt='.3f', cmap='RdYlGn', ax=ax)
    ax.set_ylabel('Resolution', fontweight='bold')
    ax.set_xlabel('Threshold Percentile', fontweight='bold')
    ax.set_title('Stability Heatmap', fontweight='bold')
    
    ax = axes[0, 1]
    ax.hist(sweep_df['stability'], bins=20, color='#0173B2', alpha=0.7, edgecolor='black')
    ax.axvline(best_params['stability'], color='red', linestyle='--', linewidth=2)
    ax.set_xlabel('Stability', fontweight='bold')
    ax.set_ylabel('Frequency', fontweight='bold')
    ax.set_title('Stability Distribution', fontweight='bold')
    ax.grid(alpha=0.3)
    
    ax = axes[0, 2]
    for thresh in sorted(sweep_df['threshold_percentile'].unique()):
        data = sweep_df[sweep_df['threshold_percentile'] == thresh]
        ax.plot(data['resolution'], data['stability'], 'o-', label=f'P{thresh}', markersize=4)
    ax.set_xlabel('Resolution', fontweight='bold')
    ax.set_ylabel('Stability', fontweight='bold')
    ax.set_title('Stability vs Resolution', fontweight='bold')
    ax.set_xscale('log')
    ax.legend(fontsize=7)
    ax.grid(alpha=0.3)
    
    ax = axes[1, 0]
    ax.scatter(sweep_df['n_clusters'], sweep_df['n_cross_clusters'],
              c=sweep_df['stability'], cmap='viridis', s=50, edgecolors='black')
    ax.scatter(best_params['n_clusters'], best_params['n_cross_clusters'],
              color='red', s=200, marker='*', edgecolors='black', linewidth=2)
    ax.set_xlabel('Total Clusters', fontweight='bold')
    ax.set_ylabel('Cross-Dataset Clusters', fontweight='bold')
    ax.set_title('Cross-Dataset vs Total', fontweight='bold')
    ax.set_xscale('log')
    ax.grid(alpha=0.3)
    
    ax = axes[1, 1]
    ax.plot(sweep_df.groupby('threshold_percentile')['n_multi_dataset_clusters'].max(), 'o-', linewidth=2)
    ax.set_xlabel('Threshold Percentile', fontweight='bold')
    ax.set_ylabel('Multi-Dataset Clusters (3+)', fontweight='bold')
    ax.set_title('Multi-Dataset Clusters', fontweight='bold')
    ax.grid(alpha=0.3)
    
    ax = axes[1, 2]
    ax.scatter(sweep_df['pct_cross_clusters'], sweep_df['stability'],
              c=sweep_df['threshold_value'], cmap='coolwarm', s=50, edgecolors='black')
    ax.scatter(best_params['pct_cross_clusters'], best_params['stability'],
              color='red', s=200, marker='*', edgecolors='black', linewidth=2)
    ax.set_xlabel('% Cross-Dataset Clusters', fontweight='bold')
    ax.set_ylabel('Stability', fontweight='bold')
    ax.set_title('Stability vs Cross-Dataset %', fontweight='bold')
    ax.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / 'poem_parameter_sweep.png', dpi=300, bbox_inches='tight')
    logger.info("✓ Visualization saved")
    plt.close()

def cluster_all_poems(poem_embeddings, poem_datasets, threshold, resolution):
    logger.info("\n" + "="*80)
    logger.info("CLUSTERING ALL POEMS")
    logger.info("="*80)
    logger.info(f"Threshold: {threshold:.4f}, Resolution: {resolution:.6e}")
    
    logger.info("Normalizing embeddings...")
    faiss.normalize_L2(poem_embeddings)
    
    logger.info("Building FAISS index...")
    dimension = poem_embeddings.shape[1]
    nlist = min(int(4 * np.sqrt(len(poem_embeddings))), 8192)
    nprobe = min(FAISS_NPROBE, nlist // 4)
    
    quantizer = faiss.IndexFlatIP(dimension)
    index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_INNER_PRODUCT)
    
    try:
        res = faiss.StandardGpuResources()
        index = faiss.index_cpu_to_gpu(res, 0, index)
        logger.info("✓ Using GPU")
    except:
        logger.info("✓ Using CPU")
    
    if len(poem_embeddings) > 1000000:
        train_size = min(500000, len(poem_embeddings))
        train_idx = np.random.choice(len(poem_embeddings), train_size, replace=False)
        index.train(poem_embeddings[train_idx])
    else:
        index.train(poem_embeddings)
    
    index.add(poem_embeddings)
    index.nprobe = nprobe
    
    logger.info(f"Searching for {N_NEIGHBORS} neighbors...")
    SEARCH_BATCH = 50000
    all_sims = []
    all_inds = []
    
    for i in tqdm(range(0, len(poem_embeddings), SEARCH_BATCH), desc="Neighbor search"):
        batch_end = min(i + SEARCH_BATCH, len(poem_embeddings))
        D, I = index.search(poem_embeddings[i:batch_end], N_NEIGHBORS)
        all_sims.append(D)
        all_inds.append(I)
    
    all_sims = np.vstack(all_sims)
    all_inds = np.vstack(all_inds)
    
    logger.info("✓ Neighbor search complete")
    
    edge_checkpoint = CHECKPOINT_DIR / 'poem_edges.npz'
    
    if edge_checkpoint.exists():
        logger.info("Loading edges from checkpoint...")
        edge_data = np.load(edge_checkpoint)
        all_edges = [(int(i), int(j)) for i, j in edge_data['edges']]
        all_weights = edge_data['weights'].tolist()
        logger.info(f"✓ Loaded {len(all_edges):,} edges")
    else:
        logger.info("Building edge list (cross-dataset only)...")
        all_edges = []
        all_weights = []
        
        EDGE_BATCH = 20000
        for batch_start in tqdm(range(0, len(poem_embeddings), EDGE_BATCH), desc="Building edges"):
            batch_end = min(batch_start + EDGE_BATCH, len(poem_embeddings))
            
            for local_idx in range(batch_end - batch_start):
                node_idx = batch_start + local_idx
                dataset_i = poem_datasets[node_idx]
                
                neighbors = all_inds[node_idx, 1:]
                sims = all_sims[node_idx, 1:]
                
                valid_mask = (neighbors > node_idx) & (sims >= threshold)
                valid_neighbors = neighbors[valid_mask]
                valid_sims = sims[valid_mask]
                
                if len(valid_neighbors) > 0:
                    neighbor_datasets = np.array([poem_datasets[n] for n in valid_neighbors])
                    cross_mask = neighbor_datasets != dataset_i
                    
                    final_neighbors = valid_neighbors[cross_mask]
                    final_sims = valid_sims[cross_mask]
                    
                    for neighbor, sim in zip(final_neighbors, final_sims):
                        all_edges.append((node_idx, int(neighbor)))
                        all_weights.append(float(sim))
            
            if batch_start > 0 and batch_start % (EDGE_BATCH * 10) == 0:
                gc.collect()
        
        logger.info(f"✓ Built {len(all_edges):,} cross-dataset edges")
        
        logger.info("Saving edge checkpoint...")
        np.savez_compressed(edge_checkpoint,
                          edges=np.array(all_edges, dtype=np.int32),
                          weights=np.array(all_weights, dtype=np.float32))
    
    logger.info("Building graph...")
    g = ig.Graph(n=len(poem_embeddings), edges=all_edges, directed=False)
    g.es['weight'] = all_weights
    
    logger.info(f"Graph: {g.vcount():,} nodes, {g.ecount():,} edges")
    
    logger.info("Running Leiden clustering...")
    partition = la.find_partition(
        g,
        la.CPMVertexPartition,
        weights='weight',
        resolution_parameter=resolution,
        n_iterations=10,
        seed=42
    )
    
    labels = np.array(partition.membership)
    
    cluster_datasets = defaultdict(set)
    cluster_poems = defaultdict(list)
    
    for idx, cid in enumerate(labels):
        cluster_datasets[cid].add(poem_datasets[idx])
        cluster_poems[cid].append(idx)
    
    n_total = len(set(labels))
    n_cross = sum(1 for ds in cluster_datasets.values() if len(ds) > 1)
    n_multi = sum(1 for ds in cluster_datasets.values() if len(ds) >= 3)
    
    logger.info(f"\nFINAL RESULTS:")
    logger.info(f"  Total clusters: {n_total:,}")
    logger.info(f"  Cross-dataset clusters: {n_cross:,} ({n_cross/n_total*100:.1f}%)")
    logger.info(f"  Multi-dataset (3+): {n_multi:,} ({n_multi/n_total*100:.1f}%)")
    
    cluster_info = []
    for cid, datasets in cluster_datasets.items():
        if len(datasets) > 1:
            poems = cluster_poems[cid]
            counts = Counter(poem_datasets[i] for i in poems)
            cluster_info.append({
                'cluster_id': cid,
                'size': len(poems),
                'n_datasets': len(datasets),
                'datasets': ', '.join(sorted(datasets)),
                'dataset_counts': dict(counts)
            })
    
    cluster_df = pd.DataFrame(cluster_info).sort_values('size', ascending=False)
    
    logger.info(f"\nTop 20 cross-dataset clusters:")
    if len(cluster_df) > 0:
        print(cluster_df.head(20).to_string(index=False))
    
    return labels, cluster_datasets, cluster_df

def main():
    start_time = time.time()
    
    logger.info("Loading verse data and embeddings...")
    df = pd.read_csv('concatenated.csv')
    
    embeddings_file = CHECKPOINT_DIR.parent / 'full_semantic_clustering_checkpoints' / 'embeddings.npz'
    if not embeddings_file.exists():
        logger.error(f"Verse embeddings not found at {embeddings_file}")
        logger.error("Please run verse-level embedding script first!")
        return
    
    verse_embeddings = np.load(embeddings_file)['embeddings']
    logger.info(f"✓ Loaded {len(verse_embeddings):,} verse embeddings")
    
    poem_embeddings, poem_ids, poem_datasets, poem_sizes = aggregate_verse_embeddings_to_poems(
        df, verse_embeddings
    )
    
    del verse_embeddings
    gc.collect()
    
    sample_indices = stratified_sample_poems(poem_datasets, poem_sizes, SAMPLE_SIZE)
    sample_embeddings = poem_embeddings[sample_indices].copy()
    sample_datasets = np.array([poem_datasets[i] for i in sample_indices])
    
    threshold, resolution = grid_search_parameters(sample_embeddings, sample_datasets)
    
    del sample_embeddings, sample_datasets
    gc.collect()
    
    poem_labels, cluster_datasets, cluster_df = cluster_all_poems(
        poem_embeddings, poem_datasets, threshold, resolution
    )
    
    poem_id_to_label = {poem_id: label for poem_id, label in zip(poem_ids, poem_labels)}
    
    df['poem_composite_id'] = df['idoriginal_poem'].astype(str) + '___' + df['source_dataset'].astype(str)
    df['poem_cluster_id'] = df['poem_composite_id'].map(poem_id_to_label)
    
    cross_cluster_ids = set(cid for cid, ds in cluster_datasets.items() if len(ds) > 1)
    df['is_cross_dataset_poem_cluster'] = df['poem_cluster_id'].isin(cross_cluster_ids)
    
    logger.info("\n" + "="*80)
    logger.info("SAVING RESULTS")
    logger.info("="*80)
    
    df.to_csv(RESULTS_DIR / 'poems_clustered_full.csv', index=False)
    logger.info(f"✓ Full results: {RESULTS_DIR / 'poems_clustered_full.csv'}")
    
    df_cross = df[df['is_cross_dataset_poem_cluster']].copy()
    df_cross.to_csv(RESULTS_DIR / 'poems_clustered_cross_dataset_only.csv', index=False)
    logger.info(f"✓ Cross-dataset only: {len(df_cross):,} verses")
    
    cluster_df.to_csv(RESULTS_DIR / 'poem_clusters_summary.csv', index=False)
    logger.info(f"✓ Cluster summary saved")
    
    summary = {
        'n_verses': len(df),
        'n_poems': len(poem_ids),
        'n_datasets': len(set(poem_datasets)),
        'threshold': threshold,
        'resolution': resolution,
        'n_total_clusters': len(set(poem_labels)),
        'n_cross_clusters': len(cross_cluster_ids),
        'n_multi_clusters': sum(1 for ds in cluster_datasets.values() if len(ds) >= 3),
        'processing_time_minutes': (time.time() - start_time) / 60
    }
    
    pd.DataFrame([summary]).to_csv(RESULTS_DIR / 'clustering_summary.csv', index=False)
    
    logger.info("\n" + "="*80)
    logger.info("✅ POEM-LEVEL CLUSTERING COMPLETE")
    logger.info("="*80)
    logger.info(f"Time: {summary['processing_time_minutes']:.1f} minutes")
    logger.info(f"Cross-dataset clusters: {summary['n_cross_clusters']:,}")
    logger.info(f"Multi-dataset (3+) clusters: {summary['n_multi_clusters']:,}")
    logger.info(f"Results saved to: {RESULTS_DIR}/")
    logger.info("="*80)

if __name__ == "__main__":
    main()

10:51:09 | INFO | ================================================================================
10:51:09 | INFO | POEM-LEVEL CLUSTERING VIA AGGREGATED EMBEDDINGS
10:51:09 | INFO | ================================================================================
10:51:09 | INFO | Loading verse data and embeddings...
/tmp/ipykernel_3761598/354242294.py:651: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('concatenated.csv')
10:51:38 | INFO | ✓ Loaded 1,536,616 verse embeddings
10:51:38 | INFO | 
10:51:38 | INFO | AGGREGATING VERSE EMBEDDINGS TO POEMS
10:51:38 | INFO | ================================================================================
10:51:40 | INFO | Found 261,216 unique poems
10:51:40 | INFO | Computing poem embeddings via mean pooling...
Aggregating:   0%|          | 0/261216 [00:00<?, ?it/s]


IndexError: index 2330186 is out of bounds for axis 0 with size 1536616